In [1]:
# Needed packages and defined functions

import numpy as np
import ab_fullerene as ab

def A_test(A):
    boo = True
    n = 2*(A.shape[0]-2)
    if np.sum(A)!=3*n:
        boo = False
        #print('Test 1 failed')
    if np.sum(np.sum(A,axis=0)==5)!=12:
        boo = False
        #print('Test 2 failed')
    if np.sum(np.sum(A,axis=0)==6)!=int(n/2-10):
        boo = False
        #print('Test 3 failed')
    return boo

def spiral_to_A(n,penta_indexes,to_print=False):
    m = int(n/2+2)
    A = np.zeros((m,m))

    # Create a counter for each vertex how many connenctions are left
    free_edge_counter = np.zeros(m)
    for i in range(m):
        if np.sum(i==penta_indexes)==1:
            free_edge_counter[i] = 5
        else:
            free_edge_counter[i] = 6
    ####print(Free_edge_counter)
    #Spiral conjecture statement (1):
    # Each new face in the spiral after the second shares an edge with both its immediate predecessor in the spiral and ....
    for i in range(m-1):
        A[i,i+1] = 1
        A[i+1,i] = 1
    free_edge_counter[0] = free_edge_counter[0]-1
    free_edge_counter[-1] = free_edge_counter[-1]-1
    free_edge_counter[1:-1] = free_edge_counter[1:-1]-2
    ####print(Free_edge_counter)


    #Spiral conjecture statement (2):
    #the first face first_open_face in the preceding spiral that still has an open edge
    new_face = 0
    open_face = 2
    left_overs_start = False
    cuts = np.ones((m,2))*m
    cuts[:,1] = np.arange(0,m)
    cuts_counter = 0
    while np.sum(free_edge_counter)>2 and not left_overs_start:

        # Basic step
        if free_edge_counter[new_face]>0 and free_edge_counter[open_face]>0 and A[new_face,open_face]==0:
            #if to_print:
                #print(new_face,open_face)
            A[new_face,open_face] = 1
            A[open_face,new_face] = 1
            free_edge_counter[new_face] -= 1
            free_edge_counter[open_face] -= 1
            open_face += 1
        #Case: 'Cut corner' 
        #new_face is complete => Check all later faces whether they are complete, and if so, 
        #connected their precursor and successor
        if free_edge_counter[new_face] == 0:
            if new_face < m-1:
                for i in range(new_face+1,m-2):
                    if free_edge_counter[i]==0 and len(np.where(free_edge_counter[0:i]>0)[0])>0 and len(np.where(free_edge_counter[i:m]>0)[0])>0:
                        # Find largest precursor and smallest successor who are both not complete:
                        #print(i)
                        max_pre = np.max(np.where(free_edge_counter[0:i]>0)[0])
                        min_suc = i + np.min(np.where(free_edge_counter[i:m]>0)[0])
                        if A[max_pre,min_suc]==0:
                            #if to_print:
                            #    print('Case: Cut corner')
                            #    print(max_pre,min_suc)
                            cuts[min_suc,:] = int(np.min([max_pre,cuts[min_suc,0]])),int(min_suc)
                            cuts_counter += 1
                            #if to_print:
                            #    print('----------------')
                            A[max_pre,min_suc] = 1
                            A[min_suc,max_pre] = 1
                            free_edge_counter[max_pre] -= 1
                            free_edge_counter[min_suc] -= 1
                            if np.sum(free_edge_counter)<3:
                                left_overs_start = True
            if not left_overs_start:
                new_face = np.min(np.where(free_edge_counter>0)[0])
                if cuts_counter > 0 and cuts[new_face,0] < m:
                    open_face = np.max(np.where(A[int(cuts[new_face,0]),:]==1)[0])
                else:
                    open_face = np.max(np.where(A[new_face-1,:]==1)[0])
    
    left_overs = np.where(free_edge_counter)[0]
    if len(left_overs)==2:
    #    if to_print:
    #        print(left_overs[0],left_overs[1])
        A[left_overs[0],left_overs[1]] = 1
        A[left_overs[1],left_overs[0]] = 1
        free_edge_counter[left_overs[0]] -= 1
        free_edge_counter[left_overs[1]] -= 1
    #if to_print:
    #    print(A_test(A))
    return A  

In [2]:
#Example: All 1812 C60 vectors of pentagon indexes
penta_indexes_matrix_C60= np.array([[0,1,2,3,4,5,26,27,28,29,30,31],
[0,1,2,3,4,6,25,27,28,29,30,31],
[0,1,2,3,4,7,24,27,28,29,30,31],
[0,1,2,3,4,11,24,25,27,28,30,31],
[0,1,2,3,4,11,24,26,27,29,30,31],
[0,1,2,3,4,12,19,27,28,29,30,31],
[0,1,2,3,4,12,21,25,28,29,30,31],
[0,1,2,3,4,12,22,25,27,29,30,31],
[0,1,2,3,4,12,23,25,27,28,30,31],
[0,1,2,3,4,12,23,26,27,29,30,31],
[0,1,2,3,4,12,24,25,27,28,29,31],
[0,1,2,3,4,12,24,26,27,28,29,30],
[0,1,2,3,4,13,22,25,26,29,30,31],
[0,1,2,3,4,13,23,25,26,28,30,31],
[0,1,2,3,4,13,24,25,26,28,29,31],
[0,1,2,3,4,13,24,26,27,29,30,31],
[0,1,2,3,4,14,19,25,28,29,30,31],
[0,1,2,3,4,14,23,25,26,27,30,31],
[0,1,2,3,4,14,24,25,26,27,29,31],
[0,1,2,3,4,18,19,21,28,29,30,31],
[0,1,2,3,4,18,19,22,27,29,30,31],
[0,1,2,3,4,18,19,23,27,28,30,31],
[0,1,2,3,4,18,19,24,27,28,29,31],
[0,1,2,3,4,18,20,22,26,29,30,31],
[0,1,2,3,4,18,20,23,26,28,30,31],
[0,1,2,3,4,18,20,24,26,28,29,31],
[0,1,2,3,4,18,21,23,26,27,30,31],
[0,1,2,3,4,19,20,22,25,29,30,31],
[0,1,2,3,4,19,20,23,25,28,30,31],
[0,1,2,3,4,19,20,23,26,29,30,31],
[0,1,2,3,4,19,20,24,25,28,29,31],
[0,1,2,3,4,19,20,24,26,28,29,30],
[0,1,2,3,4,19,20,24,27,29,30,31],
[0,1,2,3,4,19,21,23,25,27,30,31],
[0,1,2,3,4,19,21,24,26,27,29,30],
[0,1,2,3,4,19,21,24,26,28,30,31],
[0,1,2,3,4,19,22,23,26,27,30,31],
[0,1,2,3,4,19,22,24,26,27,28,30],
[0,1,2,3,4,19,22,24,26,27,29,31],
[0,1,2,3,4,19,22,25,26,27,29,30],
[0,1,2,3,4,19,23,24,26,27,28,29],
[0,1,2,3,4,20,21,24,26,27,30,31],
[0,1,2,3,6,9,22,25,28,29,30,31],
[0,1,2,3,6,9,24,27,28,29,30,31],
[0,1,2,3,6,10,21,25,28,29,30,31],
[0,1,2,3,6,10,22,25,27,29,30,31],
[0,1,2,3,6,10,23,25,27,28,30,31],
[0,1,2,3,6,10,23,26,27,29,30,31],
[0,1,2,3,6,10,24,25,27,28,29,31],
[0,1,2,3,6,10,24,26,27,28,29,30],
[0,1,2,3,6,11,22,24,27,29,30,31],
[0,1,2,3,6,11,23,24,27,28,30,31],
[0,1,2,3,6,11,23,26,28,29,30,31],
[0,1,2,3,6,12,23,24,27,29,30,31],
[0,1,2,3,6,12,23,25,28,29,30,31],
[0,1,2,3,6,16,17,25,27,28,30,31],
[0,1,2,3,6,16,17,26,27,29,30,31],
[0,1,2,3,6,16,18,24,27,28,30,31],
[0,1,2,3,6,16,18,26,28,29,30,31],
[0,1,2,3,6,16,19,24,27,29,30,31],
[0,1,2,3,6,16,19,25,28,29,30,31],
[0,1,2,3,6,16,20,24,25,28,30,31],
[0,1,2,3,6,17,18,21,28,29,30,31],
[0,1,2,3,6,17,18,22,27,29,30,31],
[0,1,2,3,6,17,18,23,27,28,30,31],
[0,1,2,3,6,17,18,24,27,28,29,31],
[0,1,2,3,6,17,19,23,27,29,30,31],
[0,1,2,3,6,17,19,24,27,28,29,30],
[0,1,2,3,6,17,20,22,25,29,30,31],
[0,1,2,3,6,17,20,23,25,28,30,31],
[0,1,2,3,6,17,20,24,25,28,29,31],
[0,1,2,3,6,17,20,24,26,28,29,30],
[0,1,2,3,6,17,20,24,27,29,30,31],
[0,1,2,3,6,17,21,24,25,27,29,31],
[0,1,2,3,6,17,21,24,26,27,29,30],
[0,1,2,3,6,17,21,24,26,28,30,31],
[0,1,2,3,6,17,22,24,25,27,28,31],
[0,1,2,3,6,17,22,24,26,27,28,30],
[0,1,2,3,6,17,22,24,26,27,29,31],
[0,1,2,3,6,17,22,25,26,27,29,30],
[0,1,2,3,6,17,22,25,26,28,29,31],
[0,1,2,3,6,17,23,24,26,27,28,29],
[0,1,2,3,6,18,19,24,28,29,30,31],
[0,1,2,3,6,18,20,24,25,28,29,30],
[0,1,2,3,6,18,21,24,25,27,29,30],
[0,1,2,3,6,18,21,24,25,28,30,31],
[0,1,2,3,6,18,21,24,26,29,30,31],
[0,1,2,3,6,18,22,24,25,27,28,30],
[0,1,2,3,6,18,22,24,25,27,29,31],
[0,1,2,3,6,18,22,25,26,29,30,31],
[0,1,2,3,6,19,21,24,25,26,29,30],
[0,1,2,3,10,11,20,21,28,29,30,31],
[0,1,2,3,10,11,20,22,27,29,30,31],
[0,1,2,3,10,11,20,23,27,28,30,31],
[0,1,2,3,10,11,21,26,28,29,30,31],
[0,1,2,3,10,11,22,26,27,28,29,31],
[0,1,2,3,10,12,20,23,27,29,30,31],
[0,1,2,3,10,12,20,24,27,28,29,30],
[0,1,2,3,10,12,21,24,27,28,30,31],
[0,1,2,3,10,12,22,26,27,29,30,31],
[0,1,2,3,10,13,15,25,27,29,30,31],
[0,1,2,3,10,13,16,25,27,28,30,31],
[0,1,2,3,10,13,17,25,27,28,29,31],
[0,1,2,3,10,13,17,26,27,28,29,30],
[0,1,2,3,10,13,18,21,28,29,30,31],
[0,1,2,3,10,13,18,22,27,29,30,31],
[0,1,2,3,10,13,18,23,27,28,30,31],
[0,1,2,3,10,13,18,24,27,28,29,31],
[0,1,2,3,10,13,19,23,27,29,30,31],
[0,1,2,3,10,13,19,24,27,28,29,30],
[0,1,2,3,10,13,20,22,25,29,30,31],
[0,1,2,3,10,13,20,23,25,28,30,31],
[0,1,2,3,10,13,20,23,26,29,30,31],
[0,1,2,3,10,13,20,24,25,28,29,31],
[0,1,2,3,10,13,20,24,26,28,29,30],
[0,1,2,3,10,13,20,24,27,29,30,31],
[0,1,2,3,10,13,21,23,25,27,30,31],
[0,1,2,3,10,13,21,24,25,27,29,31],
[0,1,2,3,10,13,21,24,26,27,29,30],
[0,1,2,3,10,13,21,24,26,28,30,31],
[0,1,2,3,10,13,22,23,26,27,30,31],
[0,1,2,3,10,13,22,24,25,27,28,31],
[0,1,2,3,10,13,22,24,26,27,28,30],
[0,1,2,3,10,13,22,24,26,27,29,31],
[0,1,2,3,10,13,22,25,26,27,29,30],
[0,1,2,3,10,13,22,25,26,28,29,31],
[0,1,2,3,10,13,23,24,26,27,28,29],
[0,1,2,3,10,14,18,21,27,29,30,31],
[0,1,2,3,10,14,18,26,27,28,29,30],
[0,1,2,3,10,14,19,23,27,28,30,31],
[0,1,2,3,10,14,19,24,27,28,29,31],
[0,1,2,3,10,14,19,25,27,28,29,30],
[0,1,2,3,10,14,20,21,25,29,30,31],
[0,1,2,3,10,14,20,24,26,28,29,31],
[0,1,2,3,10,14,20,24,27,28,30,31],
[0,1,2,3,10,14,20,25,26,28,29,30],
[0,1,2,3,10,14,20,25,27,29,30,31],
[0,1,2,3,10,14,21,25,26,27,29,30],
[0,1,2,3,10,14,21,25,26,28,29,31],
[0,1,2,3,10,15,19,21,27,29,30,31],
[0,1,2,3,10,15,19,22,27,28,30,31],
[0,1,2,3,10,15,20,21,25,28,30,31],
[0,1,2,3,10,15,20,21,26,29,30,31],
[0,1,2,3,10,15,20,22,26,28,30,31],
[0,1,2,3,10,15,20,24,27,28,29,31],
[0,1,2,3,10,15,20,25,27,28,29,30],
[0,1,2,3,10,15,20,26,27,29,30,31],
[0,1,2,3,10,15,21,22,26,27,30,31],
[0,1,2,3,10,15,21,24,26,27,28,29],
[0,1,2,3,10,16,19,21,27,28,29,30],
[0,1,2,3,10,16,19,22,27,28,29,31],
[0,1,2,3,10,16,20,21,25,28,29,31],
[0,1,2,3,10,16,20,21,26,28,29,30],
[0,1,2,3,10,16,20,21,27,29,30,31],
[0,1,2,3,10,16,20,22,26,28,29,31],
[0,1,2,3,10,17,19,22,27,28,29,30],
[0,1,2,3,10,17,20,21,22,29,30,31],
[0,1,2,3,10,17,20,21,23,28,30,31],
[0,1,2,3,10,17,20,21,24,28,29,31],
[0,1,2,3,10,17,20,22,26,28,29,30],
[0,1,2,3,10,17,20,22,27,29,30,31],
[0,1,2,3,10,17,20,23,27,28,29,30],
[0,1,2,3,10,17,21,22,24,27,28,31],
[0,1,2,3,10,17,21,22,26,27,29,30],
[0,1,2,3,10,17,21,22,26,28,29,31],
[0,1,2,3,10,18,20,21,23,29,30,31],
[0,1,2,3,10,18,20,21,24,28,29,30],
[0,1,2,3,10,18,20,22,23,28,30,31],
[0,1,2,3,10,18,20,22,24,28,29,31],
[0,1,2,3,10,18,20,22,25,28,29,30],
[0,1,2,3,10,18,20,23,27,29,30,31],
[0,1,2,3,10,18,21,22,23,27,30,31],
[0,1,2,3,10,18,21,22,24,27,28,30],
[0,1,2,3,10,18,21,22,24,27,29,31],
[0,1,2,3,10,18,21,22,25,27,29,30],
[0,1,2,3,10,18,21,22,25,28,29,31],
[0,1,2,3,10,18,21,22,26,29,30,31],
[0,1,2,3,10,18,21,23,24,27,28,29],
[0,1,2,3,10,18,21,23,26,28,29,30],
[0,1,2,3,10,19,20,21,24,29,30,31],
[0,1,2,3,10,19,20,22,24,28,30,31],
[0,1,2,3,10,19,20,22,25,29,30,31],
[0,1,2,3,10,19,20,23,24,28,29,31],
[0,1,2,3,10,19,20,23,25,28,29,30],
[0,1,2,3,10,19,20,23,26,29,30,31],
[0,1,2,3,10,19,21,22,23,26,30,31],
[0,1,2,3,10,19,21,22,24,25,28,31],
[0,1,2,3,10,19,21,22,24,26,28,30],
[0,1,2,3,10,19,21,22,24,26,29,31],
[0,1,2,3,10,19,21,22,24,27,30,31],
[0,1,2,3,10,19,21,22,25,26,29,30],
[0,1,2,3,10,19,21,23,24,26,28,29],
[0,1,2,3,10,19,21,23,24,27,29,31],
[0,1,2,3,10,19,21,23,25,27,29,30],
[0,1,2,3,10,19,22,24,27,28,30,31],
[0,1,2,3,10,20,21,22,24,25,27,31],
[0,1,2,3,10,20,21,22,24,26,27,30],
[0,1,2,3,10,20,21,22,25,26,29,31],
[0,1,2,3,10,20,21,23,24,26,27,29],
[0,1,2,3,10,20,21,23,24,26,28,31],
[0,1,2,3,10,20,21,23,25,26,28,30],
[0,1,2,3,10,20,21,24,25,28,29,31],
[0,1,2,3,10,20,22,23,25,26,28,29],
[0,1,2,3,10,20,22,23,25,27,28,31],
[0,1,2,3,10,20,22,24,25,27,28,30],
[0,1,2,3,10,20,23,24,27,28,29,31],
[0,1,2,3,11,13,17,25,27,28,29,30],
[0,1,2,3,11,13,17,26,28,29,30,31],
[0,1,2,3,11,13,18,23,27,29,30,31],
[0,1,2,3,11,13,18,24,27,28,29,30],
[0,1,2,3,11,13,20,24,26,29,30,31],
[0,1,2,3,11,13,21,23,25,26,30,31],
[0,1,2,3,11,13,21,24,25,26,29,31],
[0,1,2,3,11,13,22,24,25,26,28,31],
[0,1,2,3,11,13,22,24,26,27,30,31],
[0,1,2,3,11,13,23,24,26,27,29,31],
[0,1,2,3,11,13,23,25,26,27,29,30],
[0,1,2,3,11,13,23,25,27,28,30,31],
[0,1,2,3,11,15,18,21,27,29,30,31],
[0,1,2,3,11,15,18,22,27,28,30,31],
[0,1,2,3,11,15,21,25,26,27,29,30],
[0,1,2,3,11,18,20,21,24,29,30,31],
[0,1,2,3,11,18,20,22,24,28,30,31],
[0,1,2,3,11,18,20,22,25,29,30,31],
[0,1,2,3,11,18,20,23,24,28,29,31],
[0,1,2,3,11,18,21,22,23,26,30,31],
[0,1,2,3,11,18,21,22,24,25,28,31],
[0,1,2,3,11,18,21,22,24,26,28,30],
[0,1,2,3,11,18,21,22,24,26,29,31],
[0,1,2,3,11,18,21,22,24,27,30,31],
[0,1,2,3,11,18,21,22,25,26,29,30],
[0,1,2,3,11,18,21,23,24,26,28,29],
[0,1,2,3,11,18,21,23,24,27,29,31],
[0,1,2,3,11,18,21,23,25,27,29,30],
[0,1,2,3,11,18,22,24,27,28,30,31],
[0,1,2,3,11,19,21,22,25,29,30,31],
[0,1,2,3,11,19,21,23,25,28,30,31],
[0,1,2,3,11,19,21,23,26,29,30,31],
[0,1,2,3,11,19,21,24,25,28,29,31],
[0,1,2,3,11,19,21,24,27,29,30,31],
[0,1,2,3,11,19,22,24,27,28,30,31],
[0,1,2,3,11,19,22,25,27,29,30,31],
[0,1,2,3,11,19,23,24,27,28,29,31],
[0,1,2,3,11,20,21,24,25,27,29,31],
[0,1,2,3,11,21,22,23,25,26,27,28],
[0,1,2,3,11,21,23,24,25,27,28,29],
[0,1,2,3,12,18,21,24,25,28,29,31],
[0,1,2,3,12,19,21,24,25,27,29,31],
[0,1,2,3,12,20,24,25,27,28,29,31],
[0,1,2,3,13,17,18,20,22,29,30,31],
[0,1,2,3,13,17,18,20,23,28,30,31],
[0,1,2,3,13,17,18,20,24,28,29,31],
[0,1,2,3,13,17,18,21,23,27,30,31],
[0,1,2,3,13,17,18,22,26,27,29,30],
[0,1,2,3,13,17,18,22,26,28,29,31],
[0,1,2,3,13,17,19,23,26,27,29,30],
[0,1,2,3,13,17,19,23,27,28,30,31],
[0,1,2,3,13,18,19,20,24,29,30,31],
[0,1,2,3,13,18,19,21,23,25,30,31],
[0,1,2,3,13,18,19,21,24,25,29,31],
[0,1,2,3,13,18,19,21,24,26,29,30],
[0,1,2,3,13,18,19,22,23,26,30,31],
[0,1,2,3,13,18,19,22,24,26,28,30],
[0,1,2,3,13,18,19,22,24,26,29,31],
[0,1,2,3,13,18,19,22,24,27,30,31],
[0,1,2,3,13,18,19,22,25,26,29,30],
[0,1,2,3,13,18,19,23,24,26,28,29],
[0,1,2,3,13,18,19,23,24,27,29,31],
[0,1,2,3,13,18,19,23,25,27,29,30],
[0,1,2,3,13,18,20,21,24,26,30,31],
[0,1,2,3,13,18,20,22,24,26,27,30],
[0,1,2,3,13,18,20,22,25,26,29,31],
[0,1,2,3,13,18,20,23,24,26,27,29],
[0,1,2,3,13,18,20,23,24,26,28,31],
[0,1,2,3,13,18,20,23,25,26,28,30],
[0,1,2,3,13,18,21,22,25,26,27,30],
[0,1,2,3,13,18,21,23,24,26,27,28],
[0,1,2,3,13,19,20,23,24,26,27,31],
[0,1,2,3,18,19,20,22,23,24,26,31],
[0,1,2,3,18,19,20,22,23,25,26,30],
[0,1,2,3,18,19,20,22,24,25,29,31],
[0,1,2,3,18,19,20,23,24,25,29,30],
[0,1,2,3,18,19,20,23,24,26,29,31],
[0,1,2,3,18,19,20,23,24,27,30,31],
[0,1,2,3,18,19,21,22,23,24,26,27],
[0,1,2,3,18,19,21,22,24,25,28,31],
[0,1,2,3,18,19,21,23,24,26,28,31],
[0,1,2,3,18,19,22,23,24,27,28,31],
[0,1,2,4,6,9,22,25,27,29,30,31],
[0,1,2,4,6,9,23,25,27,28,30,31],
[0,1,2,4,6,9,23,26,27,29,30,31],
[0,1,2,4,6,9,24,26,27,28,29,30],
[0,1,2,4,6,10,22,26,28,29,30,31],
[0,1,2,4,6,10,23,26,27,28,29,30],
[0,1,2,4,6,11,22,23,27,29,30,31],
[0,1,2,4,6,15,16,25,27,29,30,31],
[0,1,2,4,6,15,17,26,28,29,30,31],
[0,1,2,4,6,15,18,23,27,29,30,31],
[0,1,2,4,6,15,19,24,28,29,30,31],
[0,1,2,4,6,15,23,24,25,27,28,31],
[0,1,2,4,6,15,23,24,26,27,28,30],
[0,1,2,4,6,15,23,24,26,27,29,31],
[0,1,2,4,6,15,24,25,26,28,29,30],
[0,1,2,4,6,16,17,26,27,28,29,30],
[0,1,2,4,6,16,18,22,27,29,30,31],
[0,1,2,4,6,16,18,23,27,28,30,31],
[0,1,2,4,6,16,19,23,27,29,30,31],
[0,1,2,4,6,16,19,24,27,28,29,30],
[0,1,2,4,6,16,20,23,25,28,30,31],
[0,1,2,4,6,16,20,23,26,29,30,31],
[0,1,2,4,6,16,20,24,25,28,29,31],
[0,1,2,4,6,16,20,24,26,28,29,30],
[0,1,2,4,6,16,20,24,27,29,30,31],
[0,1,2,4,6,16,21,23,25,27,30,31],
[0,1,2,4,6,16,21,24,25,27,29,31],
[0,1,2,4,6,16,21,24,26,27,29,30],
[0,1,2,4,6,16,21,24,26,28,30,31],
[0,1,2,4,6,16,22,23,26,27,30,31],
[0,1,2,4,6,16,22,24,25,27,28,31],
[0,1,2,4,6,16,22,24,26,27,28,30],
[0,1,2,4,6,16,22,24,26,27,29,31],
[0,1,2,4,6,16,23,24,26,27,28,29],
[0,1,2,4,6,17,19,23,28,29,30,31],
[0,1,2,4,6,17,20,23,24,28,30,31],
[0,1,2,4,6,17,21,23,24,27,30,31],
[0,1,2,4,6,17,23,25,27,28,29,30],
[0,1,2,4,6,18,23,24,26,28,29,30],
[0,1,2,4,6,19,23,24,25,27,29,30],
[0,1,2,4,9,10,20,26,27,28,29,30],
[0,1,2,4,9,11,20,22,27,29,30,31],
[0,1,2,4,9,11,21,26,28,29,30,31],
[0,1,2,4,9,11,22,26,27,28,29,31],
[0,1,2,4,9,12,20,23,27,29,30,31],
[0,1,2,4,9,12,20,24,27,28,29,30],
[0,1,2,4,9,12,21,25,28,29,30,31],
[0,1,2,4,9,12,22,25,27,28,29,31],
[0,1,2,4,9,13,16,25,27,28,30,31],
[0,1,2,4,9,13,16,26,27,29,30,31],
[0,1,2,4,9,13,17,26,27,28,29,30],
[0,1,2,4,9,13,18,22,27,29,30,31],
[0,1,2,4,9,13,18,23,27,28,30,31],
[0,1,2,4,9,13,19,23,27,29,30,31],
[0,1,2,4,9,13,19,24,27,28,29,30],
[0,1,2,4,9,13,20,22,25,29,30,31],
[0,1,2,4,9,13,20,23,25,28,30,31],
[0,1,2,4,9,13,20,23,26,29,30,31],
[0,1,2,4,9,13,20,24,25,28,29,31],
[0,1,2,4,9,13,20,24,26,28,29,30],
[0,1,2,4,9,13,20,24,27,29,30,31],
[0,1,2,4,9,13,21,23,25,27,30,31],
[0,1,2,4,9,13,21,24,25,27,29,31],
[0,1,2,4,9,13,21,24,26,27,29,30],
[0,1,2,4,9,13,21,24,26,28,30,31],
[0,1,2,4,9,13,22,23,26,27,30,31],
[0,1,2,4,9,13,22,24,25,27,28,31],
[0,1,2,4,9,13,22,24,26,27,28,30],
[0,1,2,4,9,13,22,24,26,27,29,31],
[0,1,2,4,9,13,22,25,26,27,29,30],
[0,1,2,4,9,13,22,25,26,28,29,31],
[0,1,2,4,9,13,23,24,26,27,28,29],
[0,1,2,4,9,14,18,21,27,29,30,31],
[0,1,2,4,9,14,18,26,27,28,29,30],
[0,1,2,4,9,14,19,24,27,28,29,31],
[0,1,2,4,9,14,20,21,25,29,30,31],
[0,1,2,4,9,14,20,23,26,28,30,31],
[0,1,2,4,9,14,20,24,26,28,29,31],
[0,1,2,4,9,14,20,24,27,28,30,31],
[0,1,2,4,9,14,20,25,26,28,29,30],
[0,1,2,4,9,14,20,25,27,29,30,31],
[0,1,2,4,9,14,21,23,26,27,30,31],
[0,1,2,4,9,14,21,24,25,27,28,31],
[0,1,2,4,9,14,21,24,26,27,28,30],
[0,1,2,4,9,14,21,24,26,27,29,31],
[0,1,2,4,9,14,21,25,26,27,29,30],
[0,1,2,4,9,14,21,25,26,28,29,31],
[0,1,2,4,9,14,22,25,27,28,30,31],
[0,1,2,4,9,14,22,26,27,29,30,31],
[0,1,2,4,9,15,19,21,27,29,30,31],
[0,1,2,4,9,15,20,21,25,28,30,31],
[0,1,2,4,9,15,20,21,26,29,30,31],
[0,1,2,4,9,15,20,22,26,28,30,31],
[0,1,2,4,9,15,20,24,27,28,29,31],
[0,1,2,4,9,15,20,25,27,28,29,30],
[0,1,2,4,9,15,20,26,27,29,30,31],
[0,1,2,4,9,15,21,22,26,27,30,31],
[0,1,2,4,9,15,21,24,26,27,28,29],
[0,1,2,4,9,16,19,21,27,28,29,30],
[0,1,2,4,9,16,19,22,27,28,29,31],
[0,1,2,4,9,16,20,21,25,28,29,31],
[0,1,2,4,9,16,20,21,26,28,29,30],
[0,1,2,4,9,16,20,21,27,29,30,31],
[0,1,2,4,9,16,20,22,26,28,29,31],
[0,1,2,4,9,16,20,22,27,28,30,31],
[0,1,2,4,9,16,20,23,27,28,29,31],
[0,1,2,4,9,16,21,22,25,27,28,31],
[0,1,2,4,9,16,21,22,26,27,28,30],
[0,1,2,4,9,16,21,22,26,27,29,31],
[0,1,2,4,9,16,21,23,26,27,28,29],
[0,1,2,4,9,17,20,21,22,29,30,31],
[0,1,2,4,9,17,20,21,23,28,30,31],
[0,1,2,4,9,17,20,21,24,28,29,31],
[0,1,2,4,9,17,20,22,27,29,30,31],
[0,1,2,4,9,17,20,23,27,28,29,30],
[0,1,2,4,9,17,21,22,24,27,28,31],
[0,1,2,4,9,17,21,22,26,27,29,30],
[0,1,2,4,9,18,20,21,23,29,30,31],
[0,1,2,4,9,18,20,21,24,28,29,30],
[0,1,2,4,9,18,20,22,23,28,30,31],
[0,1,2,4,9,18,20,22,24,28,29,31],
[0,1,2,4,9,18,20,23,27,29,30,31],
[0,1,2,4,9,18,21,22,23,27,30,31],
[0,1,2,4,9,18,21,22,24,27,28,30],
[0,1,2,4,9,18,21,22,24,27,29,31],
[0,1,2,4,9,18,21,22,25,27,29,30],
[0,1,2,4,9,18,21,22,26,29,30,31],
[0,1,2,4,9,18,21,23,24,27,28,29],
[0,1,2,4,9,18,21,23,26,28,29,30],
[0,1,2,4,9,19,20,21,24,29,30,31],
[0,1,2,4,9,19,20,22,24,28,30,31],
[0,1,2,4,9,19,20,22,25,29,30,31],
[0,1,2,4,9,19,20,23,24,28,29,31],
[0,1,2,4,9,19,20,23,25,28,29,30],
[0,1,2,4,9,19,20,23,26,29,30,31],
[0,1,2,4,9,19,21,22,23,26,30,31],
[0,1,2,4,9,19,21,22,24,25,28,31],
[0,1,2,4,9,19,21,22,24,26,28,30],
[0,1,2,4,9,19,21,22,24,26,29,31],
[0,1,2,4,9,19,21,22,24,27,30,31],
[0,1,2,4,9,19,21,22,25,26,29,30],
[0,1,2,4,9,19,21,23,24,26,28,29],
[0,1,2,4,9,19,21,23,24,27,29,31],
[0,1,2,4,9,19,21,23,25,27,29,30],
[0,1,2,4,9,19,22,24,27,28,30,31],
[0,1,2,4,9,20,21,22,24,25,27,31],
[0,1,2,4,9,20,21,22,24,26,27,30],
[0,1,2,4,9,20,21,22,25,26,29,31],
[0,1,2,4,9,20,21,23,24,26,27,29],
[0,1,2,4,9,20,21,23,24,26,28,31],
[0,1,2,4,9,20,21,23,25,26,28,30],
[0,1,2,4,9,20,22,23,25,26,28,29],
[0,1,2,4,9,20,22,23,25,27,28,31],
[0,1,2,4,9,20,22,24,25,27,28,30],
[0,1,2,4,9,20,23,24,27,28,29,31],
[0,1,2,4,10,13,16,24,27,28,30,31],
[0,1,2,4,10,13,16,26,28,29,30,31],
[0,1,2,4,10,13,20,23,25,29,30,31],
[0,1,2,4,10,13,20,24,25,28,29,30],
[0,1,2,4,10,13,22,23,25,27,30,31],
[0,1,2,4,10,13,22,24,25,27,28,30],
[0,1,2,4,10,13,22,24,25,27,29,31],
[0,1,2,4,10,13,22,24,26,28,30,31],
[0,1,2,4,10,13,22,25,26,29,30,31],
[0,1,2,4,10,13,23,24,25,27,28,29],
[0,1,2,4,10,13,23,24,26,28,29,31],
[0,1,2,4,10,13,23,25,26,28,29,30],
[0,1,2,4,10,13,23,26,27,29,30,31],
[0,1,2,4,10,14,17,21,27,29,30,31],
[0,1,2,4,10,14,20,24,25,28,29,31],
[0,1,2,4,10,14,21,24,25,27,28,30],
[0,1,2,4,10,14,21,24,25,27,29,31],
[0,1,2,4,10,14,21,24,26,28,30,31],
[0,1,2,4,10,14,21,25,26,29,30,31],
[0,1,2,4,10,16,20,21,25,28,29,30],
[0,1,2,4,10,16,20,22,25,28,29,31],
[0,1,2,4,10,16,21,22,25,27,28,30],
[0,1,2,4,10,16,21,22,25,27,29,31],
[0,1,2,4,10,16,21,22,26,28,30,31],
[0,1,2,4,10,17,20,21,23,29,30,31],
[0,1,2,4,10,17,20,21,24,28,29,30],
[0,1,2,4,10,17,20,22,23,28,30,31],
[0,1,2,4,10,17,20,22,24,28,29,31],
[0,1,2,4,10,17,20,23,27,29,30,31],
[0,1,2,4,10,17,21,22,23,27,30,31],
[0,1,2,4,10,17,21,22,24,27,28,30],
[0,1,2,4,10,17,21,22,24,27,29,31],
[0,1,2,4,10,17,21,22,26,29,30,31],
[0,1,2,4,10,17,21,23,24,27,28,29],
[0,1,2,4,10,18,21,23,27,29,30,31],
[0,1,2,4,10,18,21,24,27,28,29,30],
[0,1,2,4,10,19,21,22,24,26,30,31],
[0,1,2,4,10,19,21,23,24,26,29,31],
[0,1,2,4,10,19,21,23,25,26,29,30],
[0,1,2,4,10,19,21,23,25,28,30,31],
[0,1,2,4,10,19,22,23,25,28,29,31],
[0,1,2,4,10,19,22,24,25,28,29,30],
[0,1,2,4,10,19,22,24,26,28,30,31],
[0,1,2,4,10,20,21,22,24,25,26,31],
[0,1,2,4,10,20,21,23,25,27,30,31],
[0,1,2,4,10,20,21,24,25,28,30,31],
[0,1,2,4,10,20,22,23,25,26,28,31],
[0,1,2,4,10,20,22,23,25,27,29,31],
[0,1,2,4,10,20,22,24,25,26,28,30],
[0,1,2,4,10,20,22,24,25,27,29,30],
[0,1,2,4,10,20,22,24,25,28,29,31],
[0,1,2,4,10,20,22,24,26,27,30,31],
[0,1,2,4,10,20,23,24,25,28,29,30],
[0,1,2,4,10,20,23,24,26,28,29,31],
[0,1,2,4,10,21,22,23,25,26,27,29],
[0,1,2,4,10,21,22,23,25,27,28,31],
[0,1,2,4,10,21,22,24,25,27,28,30],
[0,1,2,4,10,21,23,24,27,28,29,31],
[0,1,2,4,10,22,23,24,27,28,29,30],
[0,1,2,4,10,22,23,25,27,28,29,31],
[0,1,2,4,10,22,23,26,27,28,30,31],
[0,1,2,4,11,16,21,22,25,26,29,31],
[0,1,2,4,11,20,23,24,26,27,29,31],
[0,1,2,4,12,17,23,24,27,28,29,31],
[0,1,2,4,12,18,23,24,26,28,29,31],
[0,1,2,4,12,19,23,24,26,27,29,31],
[0,1,2,4,13,14,17,18,27,29,30,31],
[0,1,2,4,13,14,17,24,25,27,28,31],
[0,1,2,4,13,14,17,24,26,27,28,30],
[0,1,2,4,13,14,17,24,26,27,29,31],
[0,1,2,4,13,14,17,25,26,27,29,30],
[0,1,2,4,13,14,17,25,26,28,29,31],
[0,1,2,4,13,14,18,24,25,27,28,30],
[0,1,2,4,13,14,18,24,25,27,29,31],
[0,1,2,4,13,14,18,24,26,28,30,31],
[0,1,2,4,13,14,18,25,26,29,30,31],
[0,1,2,4,13,15,17,18,27,28,30,31],
[0,1,2,4,13,15,17,19,27,29,30,31],
[0,1,2,4,13,15,17,20,25,28,30,31],
[0,1,2,4,13,15,17,20,26,29,30,31],
[0,1,2,4,13,15,17,21,25,27,30,31],
[0,1,2,4,13,15,17,24,26,27,28,29],
[0,1,2,4,13,15,18,20,25,29,30,31],
[0,1,2,4,13,15,18,24,25,27,28,29],
[0,1,2,4,13,15,18,24,26,28,29,31],
[0,1,2,4,13,15,18,25,26,28,29,30],
[0,1,2,4,13,15,18,26,27,29,30,31],
[0,1,2,4,13,15,19,24,25,26,28,29],
[0,1,2,4,13,15,19,24,25,27,29,31],
[0,1,2,4,13,15,20,24,25,26,27,29],
[0,1,2,4,13,15,20,24,25,26,28,31],
[0,1,2,4,13,15,21,24,25,26,27,28],
[0,1,2,4,13,15,23,25,26,27,28,29],
[0,1,2,4,13,16,17,19,27,28,29,30],
[0,1,2,4,13,16,17,20,25,28,29,31],
[0,1,2,4,13,16,17,20,26,28,29,30],
[0,1,2,4,13,16,17,20,27,29,30,31],
[0,1,2,4,13,16,17,21,25,27,29,31],
[0,1,2,4,13,16,17,21,26,27,29,30],
[0,1,2,4,13,16,17,22,25,27,28,31],
[0,1,2,4,13,16,17,22,26,27,28,30],
[0,1,2,4,13,16,17,22,26,27,29,31],
[0,1,2,4,13,16,17,23,26,27,28,29],
[0,1,2,4,13,16,18,20,25,28,29,30],
[0,1,2,4,13,16,18,22,25,27,28,30],
[0,1,2,4,13,16,18,22,25,27,29,31],
[0,1,2,4,13,16,18,22,26,28,30,31],
[0,1,2,4,13,16,18,23,25,27,28,29],
[0,1,2,4,13,16,18,23,26,28,29,31],
[0,1,2,4,13,16,18,24,27,28,29,30],
[0,1,2,4,13,16,19,22,25,26,28,30],
[0,1,2,4,13,16,19,22,25,26,29,31],
[0,1,2,4,13,16,19,22,25,27,30,31],
[0,1,2,4,13,16,19,23,25,26,28,29],
[0,1,2,4,13,16,19,23,25,27,29,31],
[0,1,2,4,13,16,22,24,26,27,28,29],
[0,1,2,4,13,17,18,20,23,29,30,31],
[0,1,2,4,13,17,18,20,24,28,29,30],
[0,1,2,4,13,17,18,21,24,27,29,30],
[0,1,2,4,13,17,18,21,24,28,30,31],
[0,1,2,4,13,17,18,22,23,27,30,31],
[0,1,2,4,13,17,18,22,24,27,28,30],
[0,1,2,4,13,17,18,22,24,27,29,31],
[0,1,2,4,13,17,18,22,25,27,29,30],
[0,1,2,4,13,17,18,22,25,28,29,31],
[0,1,2,4,13,17,18,22,26,29,30,31],
[0,1,2,4,13,17,18,23,24,27,28,29],
[0,1,2,4,13,17,18,23,26,28,29,30],
[0,1,2,4,13,17,19,21,23,25,30,31],
[0,1,2,4,13,17,19,21,24,25,29,31],
[0,1,2,4,13,17,19,21,24,26,29,30],
[0,1,2,4,13,17,19,22,23,26,30,31],
[0,1,2,4,13,17,19,22,24,25,28,31],
[0,1,2,4,13,17,19,22,24,26,28,30],
[0,1,2,4,13,17,19,22,24,26,29,31],
[0,1,2,4,13,17,19,22,24,27,30,31],
[0,1,2,4,13,17,19,22,25,26,29,30],
[0,1,2,4,13,17,19,23,24,26,28,29],
[0,1,2,4,13,17,19,23,24,27,29,31],
[0,1,2,4,13,17,19,23,25,27,29,30],
[0,1,2,4,13,17,20,21,24,26,30,31],
[0,1,2,4,13,17,20,22,24,25,27,31],
[0,1,2,4,13,17,20,22,24,26,27,30],
[0,1,2,4,13,17,20,23,24,26,27,29],
[0,1,2,4,13,17,20,23,24,26,28,31],
[0,1,2,4,13,17,20,23,25,26,28,30],
[0,1,2,4,13,17,20,24,25,28,29,31],
[0,1,2,4,13,17,21,23,24,26,27,28],
[0,1,2,4,13,17,21,24,25,26,28,29],
[0,1,2,4,13,18,19,22,24,26,30,31],
[0,1,2,4,13,18,19,23,24,26,29,31],
[0,1,2,4,13,18,19,23,25,26,29,30],
[0,1,2,4,13,18,19,23,25,28,30,31],
[0,1,2,4,13,18,20,22,24,25,26,31],
[0,1,2,4,13,18,20,23,25,27,30,31],
[0,1,2,4,13,18,20,24,25,28,30,31],
[0,1,2,4,13,18,21,23,24,26,27,31],
[0,1,2,4,13,18,21,23,25,26,27,30],
[0,1,2,4,13,18,21,24,25,27,29,31],
[0,1,2,4,13,18,22,23,25,26,27,29],
[0,1,2,4,13,18,22,23,25,27,28,31],
[0,1,2,4,13,18,22,24,25,27,28,30],
[0,1,2,4,13,18,23,24,27,28,29,31],
[0,1,2,4,13,19,22,24,25,27,28,31],
[0,1,2,4,13,19,23,24,25,26,28,29],
[0,1,2,4,13,19,23,24,25,27,28,30],
[0,1,2,4,13,19,23,24,26,27,28,31],
[0,1,2,4,14,15,17,26,27,28,29,30],
[0,1,2,4,14,15,18,25,27,28,29,30],
[0,1,2,4,14,15,19,25,26,28,29,30],
[0,1,2,4,14,15,22,25,26,27,28,29],
[0,1,2,4,14,17,18,20,22,29,30,31],
[0,1,2,4,14,17,18,21,24,28,29,31],
[0,1,2,4,14,17,18,21,25,28,29,30],
[0,1,2,4,14,17,18,22,24,27,28,29],
[0,1,2,4,14,17,18,22,26,28,29,30],
[0,1,2,4,14,17,18,23,27,28,29,30],
[0,1,2,4,14,17,19,21,22,25,30,31],
[0,1,2,4,14,17,19,21,25,28,30,31],
[0,1,2,4,14,17,19,22,24,26,28,29],
[0,1,2,4,14,17,19,22,24,27,29,31],
[0,1,2,4,14,17,19,22,25,27,29,30],
[0,1,2,4,14,17,19,23,25,28,29,31],
[0,1,2,4,14,17,19,23,26,28,29,30],
[0,1,2,4,14,17,19,23,27,29,30,31],
[0,1,2,4,14,17,20,21,24,26,29,31],
[0,1,2,4,14,17,20,21,25,26,29,30],
[0,1,2,4,14,17,20,21,25,27,30,31],
[0,1,2,4,14,17,20,22,24,26,27,29],
[0,1,2,4,14,17,20,22,24,26,28,31],
[0,1,2,4,14,17,20,22,25,26,28,30],
[0,1,2,4,14,17,20,23,25,27,29,31],
[0,1,2,4,14,17,20,23,26,27,29,30],
[0,1,2,4,14,17,20,23,26,28,30,31],
[0,1,2,4,14,17,20,24,25,28,29,30],
[0,1,2,4,14,17,20,24,26,28,29,31],
[0,1,2,4,14,17,20,24,27,28,30,31],
[0,1,2,4,14,17,21,22,24,26,27,28],
[0,1,2,4,14,17,21,22,25,26,27,29],
[0,1,2,4,14,17,21,23,25,27,28,31],
[0,1,2,4,14,17,21,23,26,27,28,30],
[0,1,2,4,14,17,21,23,26,27,29,31],
[0,1,2,4,14,17,21,24,25,27,28,29],
[0,1,2,4,14,17,22,23,26,27,28,29],
[0,1,2,4,14,18,19,22,24,26,29,31],
[0,1,2,4,14,18,19,22,25,26,29,30],
[0,1,2,4,14,18,19,22,25,28,30,31],
[0,1,2,4,14,18,19,23,26,29,30,31],
[0,1,2,4,14,18,19,24,28,29,30,31],
[0,1,2,4,14,18,20,21,25,26,30,31],
[0,1,2,4,14,18,20,22,25,27,30,31],
[0,1,2,4,14,18,20,23,25,26,29,31],
[0,1,2,4,14,18,20,24,26,28,30,31],
[0,1,2,4,14,18,20,24,27,29,30,31],
[0,1,2,4,14,18,21,22,24,26,27,31],
[0,1,2,4,14,18,21,22,25,26,27,30],
[0,1,2,4,14,18,21,23,25,26,28,31],
[0,1,2,4,14,18,21,23,26,27,30,31],
[0,1,2,4,14,18,21,24,25,26,28,29],
[0,1,2,4,14,18,21,24,25,27,29,30],
[0,1,2,4,14,18,21,24,26,27,29,31],
[0,1,2,4,14,18,21,24,27,28,30,31],
[0,1,2,4,14,18,22,23,26,27,29,31],
[0,1,2,4,14,18,22,24,27,28,29,31],
[0,1,2,4,14,19,20,24,25,26,29,30],
[0,1,2,4,14,19,20,24,26,27,30,31],
[0,1,2,4,14,19,21,24,25,26,27,29],
[0,1,2,4,14,19,21,24,25,26,28,30],
[0,1,2,4,14,19,22,24,25,26,28,29],
[0,1,2,4,14,19,22,24,25,27,28,30],
[0,1,2,4,14,19,22,24,26,27,28,31],
[0,1,2,4,14,19,23,25,26,28,29,31],
[0,1,2,4,14,19,23,25,27,28,30,31],
[0,1,2,4,14,20,21,24,25,26,27,28],
[0,1,2,4,15,18,22,23,27,28,29,31],
[0,1,2,4,15,19,22,23,26,27,28,31],
[0,1,2,4,17,18,19,22,23,24,29,31],
[0,1,2,4,17,18,20,21,23,25,30,31],
[0,1,2,4,17,18,20,22,23,24,26,29],
[0,1,2,4,17,18,20,23,24,28,30,31],
[0,1,2,4,17,18,21,22,23,24,26,28],
[0,1,2,4,17,18,21,22,23,24,27,31],
[0,1,2,4,17,18,21,22,23,25,26,29],
[0,1,2,4,17,18,21,23,24,25,28,29],
[0,1,2,4,17,18,21,23,24,27,29,31],
[0,1,2,4,17,19,20,22,23,24,26,31],
[0,1,2,4,17,19,20,22,23,25,26,30],
[0,1,2,4,17,19,20,22,24,25,29,31],
[0,1,2,4,17,19,20,23,24,25,29,30],
[0,1,2,4,17,19,20,23,24,26,29,31],
[0,1,2,4,17,19,20,23,24,27,30,31],
[0,1,2,4,17,19,21,22,23,24,26,27],
[0,1,2,4,17,19,21,22,24,25,28,31],
[0,1,2,4,17,19,21,23,24,25,27,29],
[0,1,2,4,17,19,21,23,24,25,28,30],
[0,1,2,4,17,19,22,23,24,25,28,29],
[0,1,2,4,18,19,22,23,24,25,27,29],
[0,1,2,4,18,20,22,23,24,25,27,28],
[0,1,2,6,9,11,20,24,25,28,29,31],
[0,1,2,6,9,11,21,24,25,27,29,31],
[0,1,2,6,9,11,22,24,25,27,28,31],
[0,1,2,6,11,16,18,21,26,29,30,31],
[0,1,2,6,11,16,18,22,26,28,30,31],
[0,1,2,7,17,18,22,23,27,28,30,31],
[0,1,2,9,10,11,21,22,24,27,29,30],
[0,1,2,9,10,11,21,22,24,28,30,31],
[0,1,2,9,10,11,21,23,24,28,29,31],
[0,1,2,9,10,11,21,23,25,28,29,30],
[0,1,2,9,10,11,22,23,24,27,28,29],
[0,1,2,9,10,12,21,22,24,27,30,31],
[0,1,2,9,10,12,21,22,25,29,30,31],
[0,1,2,9,10,12,21,23,24,27,29,31],
[0,1,2,9,10,12,21,23,25,27,29,30],
[0,1,2,9,10,12,22,23,24,27,28,31],
[0,1,2,9,10,12,22,23,25,27,28,30],
[0,1,2,9,10,12,22,23,25,28,29,31],
[0,1,2,9,10,12,23,24,27,28,29,31],
[0,1,2,9,10,13,15,26,27,28,30,31],
[0,1,2,9,10,13,19,22,24,27,29,30],
[0,1,2,9,10,13,19,22,24,28,30,31],
[0,1,2,9,10,13,19,23,24,28,29,31],
[0,1,2,9,10,13,19,23,25,28,29,30],
[0,1,2,9,10,13,20,22,24,27,30,31],
[0,1,2,9,10,13,20,22,25,29,30,31],
[0,1,2,9,10,13,20,23,24,27,29,31],
[0,1,2,9,10,13,20,23,25,27,29,30],
[0,1,2,9,10,13,21,24,27,28,30,31],
[0,1,2,9,10,13,21,25,28,29,30,31],
[0,1,2,9,10,13,22,23,26,27,28,30],
[0,1,2,9,10,13,22,23,26,27,29,31],
[0,1,2,9,10,13,22,24,25,27,28,29],
[0,1,2,9,10,13,22,24,26,27,29,30],
[0,1,2,9,10,13,22,24,26,28,29,31],
[0,1,2,9,10,13,23,24,26,28,29,30],
[0,1,2,9,10,14,19,20,23,29,30,31],
[0,1,2,9,10,14,19,20,24,28,29,30],
[0,1,2,9,10,14,19,21,24,27,29,30],
[0,1,2,9,10,14,19,21,24,28,30,31],
[0,1,2,9,10,14,19,22,23,27,30,31],
[0,1,2,9,10,14,19,22,24,27,28,30],
[0,1,2,9,10,14,19,22,24,27,29,31],
[0,1,2,9,10,14,19,22,25,27,29,30],
[0,1,2,9,10,14,19,22,25,28,29,31],
[0,1,2,9,10,14,19,22,26,29,30,31],
[0,1,2,9,10,14,19,23,24,27,28,29],
[0,1,2,9,10,14,20,21,24,27,30,31],
[0,1,2,9,10,14,20,21,25,29,30,31],
[0,1,2,9,10,14,20,22,25,27,29,31],
[0,1,2,9,10,14,20,22,25,28,30,31],
[0,1,2,9,10,14,20,23,24,27,28,31],
[0,1,2,9,10,19,20,22,23,24,25,31],
[0,1,2,9,10,19,20,22,24,25,29,31],
[0,1,2,9,10,19,21,22,23,24,26,31],
[0,1,2,9,10,19,21,22,23,25,26,30],
[0,1,2,9,10,19,21,22,24,25,28,31],
[0,1,2,9,11,12,14,23,25,28,30,31],
[0,1,2,9,11,12,14,23,26,29,30,31],
[0,1,2,9,11,12,14,24,26,28,29,30],
[0,1,2,9,11,12,15,22,25,28,30,31],
[0,1,2,9,11,12,15,22,26,29,30,31],
[0,1,2,9,11,12,17,22,23,28,30,31],
[0,1,2,9,11,12,21,22,24,26,30,31],
[0,1,2,9,11,12,21,23,24,26,29,31],
[0,1,2,9,11,12,21,23,25,26,29,30],
[0,1,2,9,11,12,21,23,25,27,30,31],
[0,1,2,9,11,12,22,23,24,26,27,29],
[0,1,2,9,11,12,22,23,24,26,28,31],
[0,1,2,9,11,12,22,23,25,26,28,30],
[0,1,2,9,11,12,22,24,25,28,29,31],
[0,1,2,9,11,12,23,24,25,28,29,30],
[0,1,2,9,11,12,23,24,26,28,29,31],
[0,1,2,9,11,13,14,23,25,27,30,31],
[0,1,2,9,11,13,14,24,25,27,29,31],
[0,1,2,9,11,13,14,24,26,27,29,30],
[0,1,2,9,11,13,14,24,26,28,30,31],
[0,1,2,9,11,13,15,22,25,27,30,31],
[0,1,2,9,11,13,15,24,26,28,29,31],
[0,1,2,9,11,13,15,25,26,28,29,30],
[0,1,2,9,11,13,15,25,27,28,30,31],
[0,1,2,9,11,13,18,22,24,27,29,30],
[0,1,2,9,11,13,18,22,24,28,30,31],
[0,1,2,9,11,13,18,23,24,28,29,31],
[0,1,2,9,11,13,18,23,25,28,29,30],
[0,1,2,9,11,13,19,22,23,25,30,31],
[0,1,2,9,11,13,19,22,24,25,29,31],
[0,1,2,9,11,13,19,22,24,26,29,30],
[0,1,2,9,11,13,19,23,25,28,30,31],
[0,1,2,9,11,13,22,23,24,26,27,28],
[0,1,2,9,11,13,22,23,25,26,27,29],
[0,1,2,9,11,13,23,24,25,27,28,29],
[0,1,2,9,11,14,15,19,27,29,30,31],
[0,1,2,9,11,14,15,20,25,28,30,31],
[0,1,2,9,11,14,15,20,26,29,30,31],
[0,1,2,9,11,14,15,21,25,27,30,31],
[0,1,2,9,11,14,15,24,26,27,28,29],
[0,1,2,9,11,14,16,19,27,28,29,30],
[0,1,2,9,11,14,16,20,25,28,29,31],
[0,1,2,9,11,14,16,20,26,28,29,30],
[0,1,2,9,11,14,16,20,27,29,30,31],
[0,1,2,9,11,14,16,21,25,27,29,31],
[0,1,2,9,11,14,16,21,26,27,29,30],
[0,1,2,9,11,14,16,21,26,28,30,31],
[0,1,2,9,11,14,17,20,22,29,30,31],
[0,1,2,9,11,14,17,20,23,28,30,31],
[0,1,2,9,11,14,17,20,24,28,29,31],
[0,1,2,9,11,14,17,21,23,27,30,31],
[0,1,2,9,11,14,17,21,24,27,29,31],
[0,1,2,9,11,14,17,22,24,27,28,31],
[0,1,2,9,11,14,17,22,26,27,29,30],
[0,1,2,9,11,14,17,22,26,28,29,31],
[0,1,2,9,11,14,18,20,23,29,30,31],
[0,1,2,9,11,14,18,20,24,28,29,30],
[0,1,2,9,11,14,18,21,24,27,29,30],
[0,1,2,9,11,14,18,21,24,28,30,31],
[0,1,2,9,11,14,18,22,23,27,30,31],
[0,1,2,9,11,14,18,22,24,27,28,30],
[0,1,2,9,11,14,18,22,24,27,29,31],
[0,1,2,9,11,14,18,22,25,27,29,30],
[0,1,2,9,11,14,18,22,25,28,29,31],
[0,1,2,9,11,14,18,22,26,29,30,31],
[0,1,2,9,11,14,18,23,24,27,28,29],
[0,1,2,9,11,14,18,23,26,28,29,30],
[0,1,2,9,11,14,19,21,23,25,30,31],
[0,1,2,9,11,14,19,21,24,25,29,31],
[0,1,2,9,11,14,19,21,24,26,29,30],
[0,1,2,9,11,14,19,22,23,26,30,31],
[0,1,2,9,11,14,19,22,24,25,28,31],
[0,1,2,9,11,14,19,22,24,26,28,30],
[0,1,2,9,11,14,19,22,24,26,29,31],
[0,1,2,9,11,14,19,22,24,27,30,31],
[0,1,2,9,11,14,19,22,25,26,29,30],
[0,1,2,9,11,14,19,23,24,26,28,29],
[0,1,2,9,11,14,19,23,24,27,29,31],
[0,1,2,9,11,14,19,23,25,27,29,30],
[0,1,2,9,11,14,20,21,24,26,30,31],
[0,1,2,9,11,14,20,22,24,25,27,31],
[0,1,2,9,11,14,20,22,24,26,27,30],
[0,1,2,9,11,14,20,22,25,26,29,31],
[0,1,2,9,11,14,20,23,24,26,27,29],
[0,1,2,9,11,14,20,23,24,26,28,31],
[0,1,2,9,11,14,20,23,25,26,28,30],
[0,1,2,9,11,14,20,24,25,28,29,31],
[0,1,2,9,11,14,21,22,24,26,27,31],
[0,1,2,9,11,14,21,22,25,26,27,30],
[0,1,2,9,11,14,21,22,25,26,28,31],
[0,1,2,9,11,14,21,23,24,26,27,28],
[0,1,2,9,11,14,21,23,25,26,27,29],
[0,1,2,9,11,14,21,24,25,26,28,29],
[0,1,2,9,11,15,18,21,23,27,29,30],
[0,1,2,9,11,15,18,21,23,28,30,31],
[0,1,2,9,11,15,18,22,23,27,28,30],
[0,1,2,9,11,15,18,22,23,27,29,31],
[0,1,2,9,11,15,19,21,23,25,29,31],
[0,1,2,9,11,15,19,21,23,26,29,30],
[0,1,2,9,11,15,19,21,24,29,30,31],
[0,1,2,9,11,15,19,22,23,25,28,31],
[0,1,2,9,11,15,19,22,23,26,28,30],
[0,1,2,9,11,15,19,22,23,26,29,31],
[0,1,2,9,11,15,19,22,23,27,30,31],
[0,1,2,9,11,15,19,22,24,28,30,31],
[0,1,2,9,11,15,19,22,25,28,29,31],
[0,1,2,9,11,15,19,23,24,28,29,31],
[0,1,2,9,11,15,20,21,23,26,30,31],
[0,1,2,9,11,15,20,21,25,29,30,31],
[0,1,2,9,11,15,20,22,23,25,27,31],
[0,1,2,9,11,15,20,22,23,26,27,30],
[0,1,2,9,11,15,20,22,24,27,30,31],
[0,1,2,9,11,15,20,22,25,26,29,30],
[0,1,2,9,11,15,20,22,25,27,29,31],
[0,1,2,9,11,15,20,22,25,28,30,31],
[0,1,2,9,11,15,20,23,24,27,29,31],
[0,1,2,9,11,15,20,23,25,28,29,31],
[0,1,2,9,11,15,21,22,25,26,28,30],
[0,1,2,9,11,15,21,22,25,27,28,31],
[0,1,2,9,11,15,21,23,25,26,28,29],
[0,1,2,9,11,15,21,24,26,27,29,30],
[0,1,2,9,11,15,21,24,26,28,29,31],
[0,1,2,9,11,16,19,21,23,24,29,31],
[0,1,2,9,11,16,19,22,23,24,28,31],
[0,1,2,9,11,16,19,22,24,28,29,31],
[0,1,2,9,11,16,20,21,24,29,30,31],
[0,1,2,9,11,16,20,22,23,24,27,31],
[0,1,2,9,11,16,20,22,23,26,29,31],
[0,1,2,9,11,16,20,22,24,26,29,30],
[0,1,2,9,11,16,20,22,24,27,29,31],
[0,1,2,9,11,16,20,22,24,28,30,31],
[0,1,2,9,11,16,20,22,25,29,30,31],
[0,1,2,9,11,16,20,23,24,28,29,31],
[0,1,2,9,11,16,21,22,23,26,27,30],
[0,1,2,9,11,16,21,22,23,26,28,31],
[0,1,2,9,11,16,21,22,24,26,28,30],
[0,1,2,9,11,16,21,22,24,27,28,31],
[0,1,2,9,11,16,21,22,25,28,30,31],
[0,1,2,9,11,16,21,23,24,26,28,29],
[0,1,2,9,11,16,21,23,25,27,29,30],
[0,1,2,9,11,16,21,23,25,28,29,31],
[0,1,2,9,11,17,20,22,23,25,29,31],
[0,1,2,9,11,17,20,22,24,25,29,30],
[0,1,2,9,11,17,21,22,23,24,27,31],
[0,1,2,9,11,17,21,22,23,25,27,30],
[0,1,2,9,11,17,21,22,23,25,28,31],
[0,1,2,9,11,17,21,22,23,26,30,31],
[0,1,2,9,11,17,21,22,24,25,28,30],
[0,1,2,9,11,17,21,23,24,25,28,29],
[0,1,2,9,11,17,21,23,24,26,29,30],
[0,1,2,9,11,18,20,22,23,24,25,31],
[0,1,2,9,11,18,21,22,23,24,26,31],
[0,1,2,9,11,18,21,22,23,25,26,30],
[0,1,2,9,11,18,21,22,24,25,28,31],
[0,1,2,9,11,19,21,22,23,25,26,31],
[0,1,2,9,11,19,21,22,24,25,26,30],
[0,1,2,9,11,19,21,22,24,25,27,31],
[0,1,2,9,11,19,21,23,24,25,26,29],
[0,1,2,9,11,20,21,23,24,25,26,28],
[0,1,2,9,12,15,18,20,23,27,29,30],
[0,1,2,9,12,15,18,20,23,28,30,31],
[0,1,2,9,12,15,18,22,23,27,28,31],
[0,1,2,9,12,15,19,20,23,25,29,31],
[0,1,2,9,12,15,19,23,25,27,29,30],
[0,1,2,9,12,15,19,24,27,29,30,31],
[0,1,2,9,12,16,18,22,23,27,28,30],
[0,1,2,9,12,16,19,20,23,24,29,31],
[0,1,2,9,12,16,19,22,23,26,28,30],
[0,1,2,9,12,16,19,22,23,27,30,31],
[0,1,2,9,12,16,19,23,24,27,29,30],
[0,1,2,9,12,16,20,22,23,26,27,30],
[0,1,2,9,12,16,20,22,23,26,28,31],
[0,1,2,9,12,16,20,22,24,26,28,30],
[0,1,2,9,12,16,20,22,24,27,28,31],
[0,1,2,9,12,16,20,22,25,28,30,31],
[0,1,2,9,12,16,20,23,25,27,29,30],
[0,1,2,9,12,16,21,23,26,27,29,30],
[0,1,2,9,12,16,21,23,26,28,29,31],
[0,1,2,9,12,16,21,24,26,28,29,30],
[0,1,2,9,12,16,21,24,27,28,29,31],
[0,1,2,9,12,17,19,22,23,24,28,31],
[0,1,2,9,12,17,19,22,23,25,28,30],
[0,1,2,9,12,17,20,22,23,24,27,31],
[0,1,2,9,12,17,20,22,23,25,27,30],
[0,1,2,9,12,17,20,22,23,25,28,31],
[0,1,2,9,12,17,20,22,23,26,30,31],
[0,1,2,9,12,17,20,22,24,25,28,30],
[0,1,2,9,12,17,20,23,24,25,28,29],
[0,1,2,9,12,17,20,23,24,26,29,30],
[0,1,2,9,12,17,21,23,24,27,29,31],
[0,1,2,9,12,17,21,23,25,28,29,31],
[0,1,2,9,12,17,21,24,25,28,29,30],
[0,1,2,9,12,18,19,22,23,25,30,31],
[0,1,2,9,12,18,19,23,24,25,29,30],
[0,1,2,9,12,18,20,22,23,24,26,31],
[0,1,2,9,12,18,20,22,23,25,26,30],
[0,1,2,9,12,18,21,23,24,25,28,31],
[0,1,2,9,12,18,21,23,24,26,29,31],
[0,1,2,9,12,18,21,23,25,26,29,30],
[0,1,2,9,12,18,21,24,25,28,29,31],
[0,1,2,9,12,19,20,22,23,25,26,31],
[0,1,2,9,12,19,20,22,24,25,26,30],
[0,1,2,9,12,19,20,22,24,25,27,31],
[0,1,2,9,12,19,20,23,24,25,26,29],
[0,1,2,9,12,19,21,23,24,25,27,31],
[0,1,2,9,12,19,21,23,24,26,27,30],
[0,1,2,9,12,19,21,23,25,26,29,31],
[0,1,2,9,12,19,21,24,25,26,29,30],
[0,1,2,9,12,19,21,24,25,27,29,31],
[0,1,2,9,12,19,22,23,24,25,27,28],
[0,1,2,9,12,20,21,23,24,26,27,31],
[0,1,2,9,12,20,21,23,25,26,27,30],
[0,1,2,9,12,20,21,23,25,26,28,31],
[0,1,2,9,12,20,21,24,25,26,28,30],
[0,1,2,9,12,20,21,24,25,27,28,31],
[0,1,2,9,12,21,22,24,26,27,28,30],
[0,1,2,9,12,21,22,24,26,27,29,31],
[0,1,2,9,12,21,22,25,26,27,29,30],
[0,1,2,9,12,21,22,25,26,28,29,31],
[0,1,2,9,13,14,17,19,27,28,29,31],
[0,1,2,9,13,14,17,20,26,27,29,30],
[0,1,2,9,13,14,18,19,27,29,30,31],
[0,1,2,9,13,14,18,20,23,27,30,31],
[0,1,2,9,13,14,18,20,24,27,29,31],
[0,1,2,9,13,14,18,20,25,27,29,30],
[0,1,2,9,13,14,18,20,25,28,29,31],
[0,1,2,9,13,14,18,20,26,29,30,31],
[0,1,2,9,13,14,18,21,25,27,28,30],
[0,1,2,9,13,14,18,26,27,29,30,31],
[0,1,2,9,13,14,19,20,23,26,30,31],
[0,1,2,9,13,14,19,20,24,25,28,31],
[0,1,2,9,13,14,19,20,24,26,28,30],
[0,1,2,9,13,14,19,20,24,26,29,31],
[0,1,2,9,13,14,19,20,25,26,29,30],
[0,1,2,9,13,14,19,21,24,26,28,31],
[0,1,2,9,13,14,19,21,25,26,28,30],
[0,1,2,9,13,14,19,21,25,27,30,31],
[0,1,2,9,13,14,19,23,25,27,28,29],
[0,1,2,9,13,14,19,23,26,27,29,31],
[0,1,2,9,13,14,19,24,26,27,29,30],
[0,1,2,9,13,14,20,21,25,26,27,30],
[0,1,2,9,13,14,20,21,25,26,28,31],
[0,1,2,9,13,15,19,22,27,28,30,31],
[0,1,2,9,13,15,19,23,26,27,29,30],
[0,1,2,9,13,15,19,24,27,28,29,30],
[0,1,2,9,13,17,19,23,24,27,29,30],
[0,1,2,9,13,18,19,21,23,25,30,31],
[0,1,2,9,13,18,19,22,24,28,30,31],
[0,1,2,9,13,18,19,22,25,29,30,31],
[0,1,2,9,13,18,19,23,24,26,29,30],
[0,1,2,9,13,18,20,21,23,24,26,31],
[0,1,2,9,13,18,20,21,23,25,26,30],
[0,1,2,9,13,18,20,21,24,25,28,31],
[0,1,2,9,13,18,20,22,23,26,30,31],
[0,1,2,9,13,18,20,22,24,25,28,31],
[0,1,2,9,13,18,20,22,24,26,28,30],
[0,1,2,9,13,18,22,24,27,28,30,31],
[0,1,2,9,13,19,20,21,23,25,26,31],
[0,1,2,9,13,19,20,21,24,25,26,30],
[0,1,2,9,13,19,21,23,24,25,27,28],
[0,1,2,9,13,19,22,23,25,26,28,29],
[0,1,2,9,13,19,22,23,25,27,28,31],
[0,1,2,9,13,19,22,24,25,27,28,30],
[0,1,2,9,14,15,19,21,23,27,28,31],
[0,1,2,9,14,15,19,21,25,27,29,30],
[0,1,2,9,14,15,19,22,26,27,29,30],
[0,1,2,9,14,17,18,20,22,29,30,31],
[0,1,2,9,14,17,18,20,23,28,29,30],
[0,1,2,9,14,17,18,21,23,28,29,31],
[0,1,2,9,14,17,19,20,21,24,29,30],
[0,1,2,9,14,17,19,20,22,23,30,31],
[0,1,2,9,14,17,19,20,22,24,28,30],
[0,1,2,9,14,17,19,20,22,24,29,31],
[0,1,2,9,14,17,19,20,22,25,29,30],
[0,1,2,9,14,17,19,20,23,24,28,29],
[0,1,2,9,14,17,19,21,22,24,28,31],
[0,1,2,9,14,17,19,21,22,25,28,30],
[0,1,2,9,14,17,19,21,23,25,28,29],
[0,1,2,9,14,17,19,21,23,27,30,31],
[0,1,2,9,14,17,19,22,23,27,29,31],
[0,1,2,9,14,17,19,22,24,27,29,30],
[0,1,2,9,14,17,20,21,22,24,27,31],
[0,1,2,9,14,17,20,21,22,25,27,30],
[0,1,2,9,14,17,20,21,22,25,28,31],
[0,1,2,9,14,17,20,21,22,26,30,31],
[0,1,2,9,14,17,20,21,23,24,27,28],
[0,1,2,9,14,17,20,21,23,26,28,30],
[0,1,2,9,14,17,20,21,23,26,29,31],
[0,1,2,9,14,17,20,22,23,26,28,29],
[0,1,2,9,14,18,19,20,22,24,30,31],
[0,1,2,9,14,18,19,20,23,25,29,30],
[0,1,2,9,14,18,19,21,22,25,30,31],
[0,1,2,9,14,18,19,21,23,25,28,30],
[0,1,2,9,14,18,19,21,23,25,29,31],
[0,1,2,9,14,18,19,21,23,26,30,31],
[0,1,2,9,14,18,19,21,24,25,29,30],
[0,1,2,9,14,18,19,22,23,25,28,29],
[0,1,2,9,14,18,19,22,23,26,29,31],
[0,1,2,9,14,18,19,22,24,26,29,30],
[0,1,2,9,14,18,20,21,22,24,26,31],
[0,1,2,9,14,18,20,21,22,25,26,30],
[0,1,2,9,14,18,20,21,23,24,26,28],
[0,1,2,9,14,18,20,21,23,25,26,29],
[0,1,2,9,14,18,20,21,23,25,27,30],
[0,1,2,9,14,18,20,22,23,25,27,29],
[0,1,2,9,14,18,21,22,24,27,28,31],
[0,1,2,9,14,18,21,23,26,27,30,31],
[0,1,2,9,14,19,20,21,22,25,26,31],
[0,1,2,9,14,19,20,21,23,24,26,27],
[0,1,2,9,14,19,21,22,24,25,27,28],
[0,1,2,9,15,19,21,22,24,25,28,29],
[0,1,2,9,16,17,20,21,22,24,30,31],
[0,1,2,9,16,17,20,21,23,24,28,30],
[0,1,2,9,16,18,19,22,23,24,29,31],
[0,1,2,9,16,18,20,21,22,24,25,30],
[0,1,2,9,16,19,20,21,22,24,25,31],
[0,1,2,9,17,21,22,23,24,27,28,30],
[0,1,2,9,18,19,20,22,23,24,25,26],
[0,1,2,9,18,21,22,23,24,25,27,28],
[0,1,2,10,11,12,22,24,25,27,29,31],
[0,1,2,10,11,12,23,24,25,27,29,30],
[0,1,2,10,11,14,23,24,25,27,28,29],
[0,1,2,10,11,18,21,22,24,25,26,30],
[0,1,2,10,11,18,21,23,24,25,26,29],
[0,1,2,10,11,19,22,23,25,26,29,31],
[0,1,2,10,11,19,22,24,25,26,29,30],
[0,1,2,10,12,18,20,22,24,25,26,30],
[0,1,2,10,12,18,20,23,24,25,26,29],
[0,1,2,10,12,18,21,24,25,26,29,30],
[0,1,2,10,12,18,21,24,25,27,29,31],
[0,1,2,10,12,19,23,24,25,26,28,29],
[0,1,2,10,14,19,21,24,25,26,28,29],
[0,1,2,11,12,18,19,23,24,25,26,29],
[0,1,2,11,12,18,20,23,24,25,26,28],
[0,1,2,11,14,17,18,19,21,24,29,30],
[0,1,2,11,14,17,18,19,22,23,30,31],
[0,1,2,11,14,17,18,19,22,24,28,30],
[0,1,2,11,14,17,18,19,22,24,29,31],
[0,1,2,11,14,17,18,19,22,25,29,30],
[0,1,2,11,14,17,18,20,21,24,30,31],
[0,1,2,11,14,17,18,20,22,24,27,30],
[0,1,2,11,14,17,18,20,22,25,29,31],
[0,1,2,11,14,18,19,20,23,25,26,30],
[0,1,2,12,13,18,19,20,24,26,30,31],
[0,1,2,12,18,19,20,22,24,25,26,27],
[0,1,2,12,18,19,22,23,25,26,27,29],
[0,1,3,6,8,9,24,25,26,27,28,29],
[0,1,3,6,8,10,18,26,27,28,29,30],
[0,1,3,6,8,10,19,25,27,28,29,30],
[0,1,3,6,8,10,22,25,26,27,28,30],
[0,1,3,6,8,10,23,25,26,27,28,29],
[0,1,3,6,8,11,17,26,27,28,29,30],
[0,1,3,6,8,11,19,23,27,29,30,31],
[0,1,3,6,8,11,19,24,27,28,29,30],
[0,1,3,6,8,11,20,23,25,28,30,31],
[0,1,3,6,8,11,20,23,26,29,30,31],
[0,1,3,6,8,11,20,24,26,28,29,30],
[0,1,3,6,8,11,21,23,25,27,30,31],
[0,1,3,6,8,11,21,24,25,27,29,31],
[0,1,3,6,8,11,21,24,26,27,29,30],
[0,1,3,6,8,11,21,24,26,28,30,31],
[0,1,3,6,8,11,22,24,26,27,28,30],
[0,1,3,6,8,11,22,25,26,27,29,30],
[0,1,3,6,8,11,23,24,26,27,28,29],
[0,1,3,6,8,12,19,23,25,28,30,31],
[0,1,3,6,8,12,19,23,26,29,30,31],
[0,1,3,6,8,12,19,24,26,28,29,30],
[0,1,3,6,8,12,24,25,26,28,29,30],
[0,1,3,6,8,13,19,23,25,27,30,31],
[0,1,3,6,8,13,19,24,25,27,29,31],
[0,1,3,6,8,13,19,24,26,28,30,31],
[0,1,3,6,8,13,24,25,26,27,28,29],
[0,1,3,6,8,15,19,20,25,28,30,31],
[0,1,3,6,8,15,19,20,26,29,30,31],
[0,1,3,6,8,15,19,21,25,27,30,31],
[0,1,3,6,8,16,19,20,26,28,29,30],
[0,1,3,6,8,16,19,21,26,28,30,31],
[0,1,3,6,8,16,20,25,26,28,29,30],
[0,1,3,6,8,16,21,25,26,27,28,29],
[0,1,3,6,8,17,19,20,23,28,30,31],
[0,1,3,6,8,17,19,21,23,27,30,31],
[0,1,3,6,8,17,19,22,26,28,29,31],
[0,1,3,6,8,17,20,21,26,29,30,31],
[0,1,3,6,8,17,20,22,26,28,30,31],
[0,1,3,6,8,17,20,23,26,27,28,30],
[0,1,3,6,8,17,20,23,26,28,29,31],
[0,1,3,6,8,17,20,24,26,28,29,30],
[0,1,3,6,8,17,20,25,28,29,30,31],
[0,1,3,6,8,17,21,24,26,27,28,29],
[0,1,3,6,8,17,21,25,27,28,29,31],
[0,1,3,6,8,18,19,20,23,29,30,31],
[0,1,3,6,8,18,19,20,24,28,29,30],
[0,1,3,6,8,18,19,21,24,27,29,30],
[0,1,3,6,8,18,19,21,24,28,30,31],
[0,1,3,6,8,18,19,22,23,27,30,31],
[0,1,3,6,8,18,19,22,24,27,28,30],
[0,1,3,6,8,18,19,22,24,27,29,31],
[0,1,3,6,8,18,19,22,25,27,29,30],
[0,1,3,6,8,18,19,22,25,28,29,31],
[0,1,3,6,8,18,19,23,24,27,28,29],
[0,1,3,6,8,18,20,21,24,27,30,31],
[0,1,3,6,8,18,20,21,25,29,30,31],
[0,1,3,6,8,18,20,22,25,27,29,31],
[0,1,3,6,8,18,20,22,25,28,30,31],
[0,1,3,6,8,18,20,23,24,27,28,31],
[0,1,3,6,8,18,20,23,25,27,28,30],
[0,1,3,6,8,18,20,23,25,28,29,31],
[0,1,3,6,8,18,20,23,26,27,29,30],
[0,1,3,6,8,18,20,23,26,28,30,31],
[0,1,3,6,8,18,20,24,25,28,29,30],
[0,1,3,6,8,18,21,22,25,27,28,31],
[0,1,3,6,8,18,21,22,26,27,30,31],
[0,1,3,6,8,18,21,23,26,27,28,30],
[0,1,3,6,8,18,21,23,26,27,29,31],
[0,1,3,6,8,18,21,24,25,27,28,29],
[0,1,3,6,8,18,21,24,26,27,29,30],
[0,1,3,6,8,18,21,24,26,28,29,31],
[0,1,3,6,8,18,21,25,26,29,30,31],
[0,1,3,6,8,18,22,23,26,27,28,29],
[0,1,3,6,8,18,22,25,26,28,29,30],
[0,1,3,6,8,19,20,21,24,26,30,31],
[0,1,3,6,8,19,20,22,24,25,27,31],
[0,1,3,6,8,19,20,22,24,26,27,30],
[0,1,3,6,8,19,20,23,24,26,27,29],
[0,1,3,6,8,19,21,23,25,26,27,29],
[0,1,3,6,8,19,21,24,25,26,28,29],
[0,1,3,6,8,20,21,23,25,26,27,28],
[0,1,3,6,9,11,16,26,27,28,29,30],
[0,1,3,6,9,11,20,23,24,28,30,31],
[0,1,3,6,9,11,21,23,24,27,30,31],
[0,1,3,6,9,11,21,23,26,29,30,31],
[0,1,3,6,9,11,22,23,26,28,30,31],
[0,1,3,6,9,11,22,24,26,27,29,30],
[0,1,3,6,9,11,22,24,26,28,29,31],
[0,1,3,6,9,11,22,25,27,29,30,31],
[0,1,3,6,9,11,23,25,27,28,29,30],
[0,1,3,6,9,13,19,23,24,27,30,31],
[0,1,3,6,9,13,19,23,26,29,30,31],
[0,1,3,6,9,13,24,25,27,28,29,31],
[0,1,3,6,9,13,25,26,27,29,30,31],
[0,1,3,6,9,14,19,23,26,28,30,31],
[0,1,3,6,9,14,25,26,27,28,29,30],
[0,1,3,6,9,15,18,26,27,28,29,30],
[0,1,3,6,9,15,19,20,24,28,30,31],
[0,1,3,6,9,15,19,21,26,29,30,31],
[0,1,3,6,9,15,19,22,26,28,30,31],
[0,1,3,6,9,15,19,25,27,28,29,30],
[0,1,3,6,9,15,20,24,26,27,28,30],
[0,1,3,6,9,15,20,24,26,28,29,31],
[0,1,3,6,9,16,19,20,23,28,30,31],
[0,1,3,6,9,16,19,21,23,27,30,31],
[0,1,3,6,9,16,19,22,26,28,29,31],
[0,1,3,6,9,16,20,21,26,29,30,31],
[0,1,3,6,9,16,20,22,26,28,30,31],
[0,1,3,6,9,16,20,23,26,27,28,30],
[0,1,3,6,9,16,20,23,26,28,29,31],
[0,1,3,6,9,16,20,25,28,29,30,31],
[0,1,3,6,9,16,21,24,26,27,28,29],
[0,1,3,6,9,16,21,25,27,28,29,31],
[0,1,3,6,9,18,19,22,25,29,30,31],
[0,1,3,6,9,18,20,22,24,27,30,31],
[0,1,3,6,9,18,20,23,24,27,29,31],
[0,1,3,6,9,18,20,23,25,27,29,30],
[0,1,3,6,9,18,20,23,25,28,30,31],
[0,1,3,6,9,18,21,23,25,27,28,30],
[0,1,3,6,9,18,21,24,25,27,29,30],
[0,1,3,6,9,18,22,23,25,27,28,29],
[0,1,3,6,9,19,20,22,24,26,30,31],
[0,1,3,6,9,19,20,23,24,25,27,29],
[0,1,3,6,9,19,20,23,24,26,29,31],
[0,1,3,6,9,19,20,23,25,26,29,30],
[0,1,3,6,9,19,21,22,24,25,27,31],
[0,1,3,6,9,19,21,23,24,26,28,31],
[0,1,3,6,9,19,21,23,25,26,28,30],
[0,1,3,6,9,19,21,23,25,26,29,31],
[0,1,3,6,9,19,21,23,26,27,30,31],
[0,1,3,6,9,19,21,24,25,26,29,30],
[0,1,3,6,9,19,22,23,25,26,28,29],
[0,1,3,6,9,19,22,23,26,27,29,31],
[0,1,3,6,9,19,22,24,26,27,29,30],
[0,1,3,6,9,20,21,23,25,26,28,31],
[0,1,3,6,9,20,21,24,25,26,28,30],
[0,1,3,6,9,20,21,24,26,27,30,31],
[0,1,3,6,9,20,22,23,26,27,28,31],
[0,1,3,6,9,20,22,24,25,26,28,29],
[0,1,3,6,9,20,22,24,26,27,28,30],
[0,1,3,6,9,20,22,24,26,27,29,31],
[0,1,3,6,9,20,23,24,26,27,29,30],
[0,1,3,6,10,14,19,23,24,27,30,31],
[0,1,3,6,10,14,24,25,26,28,29,30],
[0,1,3,6,10,15,17,26,27,28,29,30],
[0,1,3,6,10,15,19,21,25,29,30,31],
[0,1,3,6,10,15,19,22,24,27,30,31],
[0,1,3,6,10,15,19,24,26,28,29,30],
[0,1,3,6,10,15,20,24,25,27,28,30],
[0,1,3,6,10,15,20,24,26,27,29,30],
[0,1,3,6,10,15,21,24,26,27,28,30],
[0,1,3,6,10,15,21,24,26,27,29,31],
[0,1,3,6,10,15,21,25,26,28,30,31],
[0,1,3,6,10,16,19,20,24,28,29,30],
[0,1,3,6,10,16,19,21,24,27,29,30],
[0,1,3,6,10,16,19,21,24,28,30,31],
[0,1,3,6,10,16,19,22,23,27,30,31],
[0,1,3,6,10,16,19,22,24,27,28,30],
[0,1,3,6,10,16,19,22,24,27,29,31],
[0,1,3,6,10,16,19,23,24,27,28,29],
[0,1,3,6,10,16,19,23,26,28,29,30],
[0,1,3,6,10,16,20,23,24,27,28,31],
[0,1,3,6,10,16,20,23,25,27,28,30],
[0,1,3,6,10,16,20,23,25,28,29,31],
[0,1,3,6,10,16,20,23,26,27,29,30],
[0,1,3,6,10,16,20,23,26,28,30,31],
[0,1,3,6,10,19,21,23,24,25,26,28],
[0,1,3,6,10,19,21,23,24,25,27,31],
[0,1,3,6,10,19,22,24,26,27,30,31],
[0,1,3,6,11,15,16,19,27,29,30,31],
[0,1,3,6,11,15,16,20,25,28,30,31],
[0,1,3,6,11,15,16,20,26,29,30,31],
[0,1,3,6,11,15,16,21,25,27,30,31],
[0,1,3,6,11,15,16,24,26,27,28,29],
[0,1,3,6,11,15,17,21,24,27,30,31],
[0,1,3,6,11,15,17,21,26,29,30,31],
[0,1,3,6,11,15,17,22,26,28,30,31],
[0,1,3,6,11,15,17,25,27,28,29,30],
[0,1,3,6,11,15,18,22,24,27,30,31],
[0,1,3,6,11,15,18,23,27,29,30,31],
[0,1,3,6,11,15,18,24,26,28,29,30],
[0,1,3,6,11,15,19,23,25,28,30,31],
[0,1,3,6,11,15,19,24,25,27,29,30],
[0,1,3,6,11,16,17,20,23,28,30,31],
[0,1,3,6,11,16,17,21,23,27,30,31],
[0,1,3,6,11,16,17,21,24,27,29,31],
[0,1,3,6,11,16,17,22,26,27,29,30],
[0,1,3,6,11,16,17,22,26,28,29,31],
[0,1,3,6,11,16,18,21,24,27,29,30],
[0,1,3,6,11,16,18,21,24,28,30,31],
[0,1,3,6,11,16,18,22,23,27,30,31],
[0,1,3,6,11,16,18,22,24,27,28,30],
[0,1,3,6,11,16,18,22,24,27,29,31],
[0,1,3,6,11,16,18,22,25,27,29,30],
[0,1,3,6,11,16,18,22,25,28,29,31],
[0,1,3,6,11,16,18,22,26,29,30,31],
[0,1,3,6,11,16,18,23,24,27,28,29],
[0,1,3,6,11,16,18,23,26,28,29,30],
[0,1,3,6,11,16,19,22,23,26,30,31],
[0,1,3,6,11,16,19,22,24,26,28,30],
[0,1,3,6,11,16,19,22,24,26,29,31],
[0,1,3,6,11,16,19,22,24,27,30,31],
[0,1,3,6,11,16,19,22,25,26,29,30],
[0,1,3,6,11,16,19,23,24,26,28,29],
[0,1,3,6,11,16,19,23,24,27,29,31],
[0,1,3,6,11,16,19,23,25,27,29,30],
[0,1,3,6,11,16,20,21,24,26,30,31],
[0,1,3,6,11,16,20,22,24,26,27,30],
[0,1,3,6,11,16,20,23,24,26,27,29],
[0,1,3,6,11,16,20,23,24,26,28,31],
[0,1,3,6,11,16,20,23,25,26,28,30],
[0,1,3,6,11,16,21,23,24,26,27,28],
[0,1,3,6,11,16,21,23,25,26,27,29],
[0,1,3,6,11,17,19,22,24,25,29,31],
[0,1,3,6,11,17,19,23,24,25,28,29],
[0,1,3,6,11,17,20,22,24,26,30,31],
[0,1,3,6,11,17,20,23,24,25,27,29],
[0,1,3,6,11,17,20,23,24,25,28,31],
[0,1,3,6,11,17,20,23,24,26,29,31],
[0,1,3,6,11,17,20,23,25,26,29,30],
[0,1,3,6,11,17,21,23,24,25,27,28],
[0,1,3,6,11,17,21,23,24,26,28,31],
[0,1,3,6,11,17,21,23,25,26,28,30],
[0,1,3,6,11,17,21,23,26,27,30,31],
[0,1,3,6,11,17,21,24,25,26,29,30],
[0,1,3,6,11,17,22,23,25,26,28,29],
[0,1,3,6,11,17,22,23,26,27,29,31],
[0,1,3,6,11,17,22,24,26,27,29,30],
[0,1,3,6,11,18,19,23,24,25,29,31],
[0,1,3,6,11,18,21,23,24,25,26,28],
[0,1,3,6,11,18,21,23,24,25,27,31],
[0,1,3,6,11,18,22,24,26,27,30,31],
[0,1,3,6,13,18,23,25,27,28,29,31],
[0,1,3,6,15,18,19,20,23,25,29,31],
[0,1,3,6,16,17,19,20,21,24,29,30],
[0,1,3,6,16,17,19,20,22,23,30,31],
[0,1,3,6,16,17,19,20,22,24,28,30],
[0,1,3,6,16,17,19,20,22,24,29,31],
[0,1,3,6,16,17,19,20,22,25,29,30],
[0,1,3,6,16,17,19,20,23,24,28,29],
[0,1,3,6,16,17,19,21,22,24,28,31],
[0,1,3,6,16,17,19,21,22,25,28,30],
[0,1,3,6,16,17,19,21,23,25,28,29],
[0,1,3,6,16,17,19,21,23,27,30,31],
[0,1,3,6,16,17,19,22,23,27,29,31],
[0,1,3,6,16,17,19,22,24,27,29,30],
[0,1,3,6,16,17,20,21,22,25,27,30],
[0,1,3,6,16,17,20,21,22,25,28,31],
[0,1,3,6,16,17,20,21,23,24,27,28],
[0,1,3,6,16,18,19,20,22,24,30,31],
[0,1,3,6,16,18,19,20,23,24,29,31],
[0,1,3,6,16,18,19,20,23,25,29,30],
[0,1,3,6,16,18,19,21,23,24,28,31],
[0,1,3,6,16,18,19,21,23,25,28,30],
[0,1,3,6,16,18,19,21,23,25,29,31],
[0,1,3,6,16,18,19,21,23,26,30,31],
[0,1,3,6,16,18,19,21,24,25,29,30],
[0,1,3,6,16,18,19,22,23,25,28,29],
[0,1,3,6,16,18,19,22,23,26,29,31],
[0,1,3,6,16,18,19,22,24,26,29,30],
[0,1,3,6,16,18,20,21,23,25,27,30],
[0,1,3,6,16,18,20,22,23,25,27,29],
[0,1,3,6,16,18,21,23,26,27,30,31],
[0,1,3,6,17,18,20,22,23,25,26,29],
[0,1,3,6,17,18,20,22,23,25,28,31],
[0,1,3,6,17,18,20,22,24,25,28,30],
[0,1,3,6,17,19,20,22,24,25,27,30],
[0,1,3,6,18,19,20,23,24,25,26,29],
[0,1,3,6,18,19,20,23,24,25,27,30],
[0,1,3,7,8,9,25,26,27,28,29,30],
[0,1,3,7,8,11,19,22,27,29,30,31],
[0,1,3,7,8,11,21,22,25,27,30,31],
[0,1,3,7,8,11,21,25,26,28,29,30],
[0,1,3,7,8,11,21,25,27,28,30,31],
[0,1,3,7,8,11,23,26,27,28,29,30],
[0,1,3,7,8,12,21,25,26,27,29,30],
[0,1,3,7,8,13,18,26,27,28,30,31],
[0,1,3,7,8,13,19,22,25,27,30,31],
[0,1,3,7,8,13,19,25,26,28,29,30],
[0,1,3,7,8,18,19,21,23,28,30,31],
[0,1,3,7,8,18,20,21,23,27,30,31],
[0,1,3,7,8,18,20,22,26,29,30,31],
[0,1,3,7,8,18,21,22,26,28,30,31],
[0,1,3,7,8,18,21,23,26,27,29,30],
[0,1,3,7,8,18,21,23,26,28,29,31],
[0,1,3,7,11,12,22,23,25,27,30,31],
[0,1,3,7,11,12,23,24,25,27,29,31],
[0,1,3,7,11,12,23,24,26,27,29,30],
[0,1,3,7,11,13,16,22,25,27,30,31],
[0,1,3,7,11,13,16,24,26,28,29,31],
[0,1,3,7,11,13,16,25,26,28,29,30],
[0,1,3,7,11,13,16,25,27,28,30,31],
[0,1,3,7,11,13,22,24,25,26,27,29],
[0,1,3,7,11,13,23,24,26,27,28,30],
[0,1,3,7,11,13,23,25,26,27,29,30],
[0,1,3,7,11,14,16,19,27,29,30,31],
[0,1,3,7,11,14,16,21,25,27,30,31],
[0,1,3,7,11,14,16,24,26,27,28,29],
[0,1,3,7,11,14,17,20,24,28,30,31],
[0,1,3,7,11,14,17,25,27,28,29,30],
[0,1,3,7,11,14,19,22,24,26,30,31],
[0,1,3,7,11,14,19,24,25,27,29,30],
[0,1,3,7,11,14,20,24,25,26,28,30],
[0,1,3,7,11,14,21,24,25,26,27,29],
[0,1,3,7,11,16,18,21,23,28,30,31],
[0,1,3,7,11,16,18,22,23,27,28,30],
[0,1,3,7,11,16,19,21,23,25,29,31],
[0,1,3,7,11,16,19,22,23,25,28,31],
[0,1,3,7,11,16,19,22,23,26,28,30],
[0,1,3,7,11,16,19,22,23,27,30,31],
[0,1,3,7,11,16,19,22,24,28,30,31],
[0,1,3,7,11,16,19,22,25,28,29,31],
[0,1,3,7,11,16,20,22,23,25,27,31],
[0,1,3,7,11,16,20,22,23,26,27,30],
[0,1,3,7,11,16,20,22,24,27,30,31],
[0,1,3,7,11,16,20,22,25,26,29,30],
[0,1,3,7,11,16,20,22,25,27,29,31],
[0,1,3,7,11,16,20,23,24,27,29,31],
[0,1,3,7,11,16,21,22,25,26,28,30],
[0,1,3,7,11,16,21,22,25,27,28,31],
[0,1,3,7,11,16,21,23,25,26,28,29],
[0,1,3,7,11,16,21,24,26,27,29,30],
[0,1,3,7,11,16,21,24,26,28,29,31],
[0,1,3,7,11,17,20,22,23,26,30,31],
[0,1,3,7,11,17,20,23,24,28,30,31],
[0,1,3,7,11,17,21,22,23,25,27,31],
[0,1,3,7,11,17,21,22,24,27,30,31],
[0,1,3,7,11,17,21,23,24,27,28,30],
[0,1,3,7,11,17,21,23,24,27,29,31],
[0,1,3,7,11,17,21,24,25,27,29,30],
[0,1,3,7,11,17,21,24,25,28,29,31],
[0,1,3,7,11,17,22,23,24,27,28,29],
[0,1,3,7,11,18,21,22,24,26,30,31],
[0,1,3,7,11,18,21,23,24,26,28,30],
[0,1,3,7,11,18,21,23,24,26,29,31],
[0,1,3,7,11,18,21,23,24,27,30,31],
[0,1,3,7,11,18,21,24,25,26,29,30],
[0,1,3,7,11,18,22,23,24,26,28,29],
[0,1,3,7,11,18,22,23,24,27,29,31],
[0,1,3,7,11,18,22,24,25,27,29,30],
[0,1,3,7,11,19,21,23,24,26,27,30],
[0,1,3,7,11,19,22,23,24,26,27,29],
[0,1,3,7,11,19,22,23,24,26,28,31],
[0,1,3,7,11,19,22,24,25,26,28,30],
[0,1,3,7,11,20,22,23,24,26,27,28],
[0,1,3,7,12,16,18,20,23,28,30,31],
[0,1,3,7,12,16,19,20,23,25,29,31],
[0,1,3,7,12,16,19,22,23,26,28,31],
[0,1,3,7,12,16,19,23,25,27,29,30],
[0,1,3,7,12,16,19,24,27,29,30,31],
[0,1,3,7,12,16,20,22,25,26,28,30],
[0,1,3,7,12,16,20,22,25,27,28,31],
[0,1,3,7,12,16,20,24,26,27,29,30],
[0,1,3,7,12,16,21,23,26,27,28,30],
[0,1,3,7,12,16,21,25,27,28,29,31],
[0,1,3,7,12,17,19,22,23,25,28,31],
[0,1,3,7,12,17,19,23,24,28,29,31],
[0,1,3,7,12,17,20,22,23,25,27,31],
[0,1,3,7,12,17,20,22,24,27,30,31],
[0,1,3,7,12,17,20,23,24,27,28,30],
[0,1,3,7,12,17,20,23,24,27,29,31],
[0,1,3,7,12,17,20,24,25,27,29,30],
[0,1,3,7,12,17,21,23,25,27,28,30],
[0,1,3,7,12,17,21,23,25,27,29,31],
[0,1,3,7,12,17,21,23,26,28,30,31],
[0,1,3,7,12,18,20,22,24,26,30,31],
[0,1,3,7,12,18,20,23,24,26,28,30],
[0,1,3,7,12,18,20,23,24,26,29,31],
[0,1,3,7,12,18,20,24,25,26,29,30],
[0,1,3,7,12,19,20,23,24,26,27,30],
[0,1,3,7,12,19,22,24,25,26,28,29],
[0,1,3,7,12,19,22,24,25,27,28,31],
[0,1,3,7,12,21,23,24,26,27,28,29],
[0,1,3,7,16,17,19,21,22,24,28,30],
[0,1,3,7,16,17,19,21,23,24,28,29],
[0,1,3,7,16,17,20,21,22,24,27,30],
[0,1,3,7,16,17,20,21,22,24,28,31],
[0,1,3,7,16,18,19,21,23,24,29,31],
[0,1,3,7,16,18,19,22,23,24,28,29],
[0,1,3,7,16,18,20,21,22,24,26,30],
[0,1,3,7,16,19,20,21,23,25,27,30],
[0,1,3,7,16,19,20,22,23,25,27,29],
[0,1,3,7,16,19,21,22,24,25,28,29],
[0,1,3,7,17,19,20,21,23,24,25,31],
[0,1,3,7,17,19,20,21,23,25,26,30],
[0,1,3,7,17,19,20,22,23,24,27,31],
[0,1,3,7,17,19,20,22,23,25,26,29],
[0,1,3,7,17,21,22,23,24,27,28,29],
[0,1,3,7,18,21,22,23,25,26,28,29],
[0,1,3,7,18,21,22,23,25,27,28,30],
[0,1,3,8,11,12,21,23,24,26,29,31],
[0,1,3,8,11,12,21,23,25,26,29,30],
[0,1,3,8,11,12,22,23,24,26,28,31],
[0,1,3,8,11,12,23,24,25,28,29,30],
[0,1,3,8,11,13,14,24,25,27,29,31],
[0,1,3,8,11,13,14,24,26,27,29,30],
[0,1,3,8,11,13,14,24,26,28,30,31],
[0,1,3,8,11,13,15,25,26,28,29,30],
[0,1,3,8,11,13,15,25,27,28,30,31],
[0,1,3,8,11,13,16,22,25,27,29,31],
[0,1,3,8,11,13,16,22,26,27,29,30],
[0,1,3,8,11,13,16,22,26,28,30,31],
[0,1,3,8,11,13,20,23,24,26,29,31],
[0,1,3,8,11,13,20,23,25,26,29,30],
[0,1,3,8,11,13,20,23,25,27,30,31],
[0,1,3,8,11,13,22,23,24,26,27,28],
[0,1,3,8,11,13,22,23,25,26,27,29],
[0,1,3,8,11,13,22,24,25,26,28,29],
[0,1,3,8,11,14,16,20,26,28,29,30],
[0,1,3,8,11,14,16,21,25,27,29,31],
[0,1,3,8,11,14,16,21,26,27,29,30],
[0,1,3,8,11,14,16,21,26,28,30,31],
[0,1,3,8,11,14,16,22,26,27,28,30],
[0,1,3,8,11,14,17,20,23,28,30,31],
[0,1,3,8,11,14,17,21,23,27,30,31],
[0,1,3,8,11,14,17,21,24,27,29,31],
[0,1,3,8,11,14,18,21,24,27,29,30],
[0,1,3,8,11,14,18,21,24,28,30,31],
[0,1,3,8,11,14,18,22,24,27,28,30],
[0,1,3,8,11,14,18,22,24,27,29,31],
[0,1,3,8,11,14,18,23,24,27,28,29],
[0,1,3,8,11,14,19,21,23,25,30,31],
[0,1,3,8,11,14,19,21,24,25,29,31],
[0,1,3,8,11,14,19,21,24,26,29,30],
[0,1,3,8,11,14,19,22,24,25,28,31],
[0,1,3,8,11,14,19,22,24,26,28,30],
[0,1,3,8,11,14,19,22,24,26,29,31],
[0,1,3,8,11,14,19,22,24,27,30,31],
[0,1,3,8,11,14,19,23,24,26,28,29],
[0,1,3,8,11,14,19,23,24,27,29,31],
[0,1,3,8,11,14,20,22,24,25,27,31],
[0,1,3,8,11,14,20,22,24,26,27,30],
[0,1,3,8,11,14,20,23,24,26,27,29],
[0,1,3,8,11,14,20,23,24,26,28,31],
[0,1,3,8,11,14,20,23,25,26,28,30],
[0,1,3,8,11,14,21,22,25,26,27,30],
[0,1,3,8,11,14,21,23,24,26,27,28],
[0,1,3,8,11,14,21,23,25,26,27,29],
[0,1,3,8,11,15,18,22,23,27,28,30],
[0,1,3,8,11,15,19,21,23,25,29,31],
[0,1,3,8,11,15,19,22,23,25,28,31],
[0,1,3,8,11,15,19,22,23,26,28,30],
[0,1,3,8,11,15,19,22,24,28,30,31],
[0,1,3,8,11,15,19,22,25,28,29,31],
[0,1,3,8,11,15,20,22,23,25,27,31],
[0,1,3,8,11,15,20,22,23,26,27,30],
[0,1,3,8,11,15,20,22,24,27,30,31],
[0,1,3,8,11,15,20,22,25,26,29,30],
[0,1,3,8,11,15,20,22,25,27,29,31],
[0,1,3,8,11,15,20,22,25,28,30,31],
[0,1,3,8,11,15,20,23,24,27,29,31],
[0,1,3,8,11,15,21,22,25,26,28,30],
[0,1,3,8,11,15,21,22,25,27,28,31],
[0,1,3,8,11,15,21,23,25,26,28,29],
[0,1,3,8,11,15,21,24,26,27,29,30],
[0,1,3,8,11,15,21,24,26,28,29,31],
[0,1,3,8,11,16,19,22,23,24,28,31],
[0,1,3,8,11,16,19,22,24,28,29,31],
[0,1,3,8,11,16,20,22,23,24,27,31],
[0,1,3,8,11,16,20,22,24,26,29,30],
[0,1,3,8,11,16,20,22,24,27,29,31],
[0,1,3,8,11,16,21,22,23,26,28,31],
[0,1,3,8,11,16,21,22,24,26,28,30],
[0,1,3,8,11,16,21,22,24,27,28,31],
[0,1,3,8,11,16,21,22,25,28,30,31],
[0,1,3,8,11,16,21,23,24,26,28,29],
[0,1,3,8,11,17,20,22,24,25,29,30],
[0,1,3,8,11,17,21,22,23,25,28,31],
[0,1,3,8,11,17,21,22,24,25,28,30],
[0,1,3,8,11,17,21,23,24,25,28,29],
[0,1,3,8,12,14,16,20,26,27,29,30],
[0,1,3,8,12,14,16,20,26,28,30,31],
[0,1,3,8,12,14,18,20,24,28,30,31],
[0,1,3,8,12,14,18,23,25,27,28,29],
[0,1,3,8,12,14,19,20,23,25,30,31],
[0,1,3,8,12,14,19,20,24,25,29,31],
[0,1,3,8,12,14,19,20,24,26,29,30],
[0,1,3,8,12,14,19,23,25,26,28,29],
[0,1,3,8,12,14,19,23,25,27,28,30],
[0,1,3,8,12,14,19,23,25,27,29,31],
[0,1,3,8,12,14,20,23,25,26,27,29],
[0,1,3,8,12,14,21,24,26,27,28,30],
[0,1,3,8,12,15,20,22,25,27,28,31],
[0,1,3,8,12,15,21,23,26,27,28,30],
[0,1,3,8,12,16,20,22,23,26,27,30],
[0,1,3,8,12,16,20,22,23,26,28,31],
[0,1,3,8,12,16,20,22,24,27,28,31],
[0,1,3,8,12,16,21,23,26,28,29,31],
[0,1,3,8,12,20,21,23,25,26,27,30],
[0,1,3,8,14,15,19,20,21,25,29,31],
[0,1,3,8,14,15,19,20,21,26,29,30],
[0,1,3,8,14,15,19,20,22,26,29,31],
[0,1,3,8,14,15,19,21,22,26,28,31],
[0,1,3,8,14,15,19,21,25,27,29,30],
[0,1,3,8,14,15,19,22,26,27,29,30],
[0,1,3,8,14,16,18,21,22,27,28,30],
[0,1,3,8,14,16,19,20,21,23,30,31],
[0,1,3,8,14,16,19,20,21,24,29,31],
[0,1,3,8,14,16,19,21,22,26,28,30],
[0,1,3,8,14,16,19,21,22,27,30,31],
[0,1,3,8,14,16,19,21,23,27,28,30],
[0,1,3,8,14,16,19,21,23,27,29,31],
[0,1,3,8,14,16,19,21,24,27,29,30],
[0,1,3,8,14,16,20,21,22,26,27,30],
[0,1,3,8,14,17,20,21,22,24,27,31],
[0,1,3,8,14,17,20,21,22,25,27,30],
[0,1,3,8,14,17,20,21,22,26,30,31],
[0,1,3,8,14,17,20,21,23,26,28,30],
[0,1,3,8,14,18,20,21,22,24,26,31],
[0,1,3,8,14,18,20,21,22,25,26,30],
[0,1,3,8,14,18,20,22,23,25,27,29],
[0,1,3,8,14,19,21,22,24,25,27,28],
[0,1,3,8,15,16,20,21,22,27,30,31],
[0,1,3,8,15,17,20,21,22,24,27,30],
[0,1,3,9,15,16,20,21,22,26,30,31],
[0,1,3,9,15,17,20,21,22,24,26,30],
[0,1,3,11,12,13,17,24,25,27,29,31],
[0,1,3,11,12,14,16,17,25,27,29,31],
[0,1,3,11,12,14,16,17,26,27,29,30],
[0,1,3,11,12,14,17,19,24,25,29,31],
[0,1,3,11,12,15,17,24,26,27,29,30],
[0,1,3,11,12,15,18,24,25,28,30,31],
[0,1,3,11,12,16,17,22,24,27,28,31],
[0,1,3,11,12,16,18,24,25,29,30,31],
[0,1,3,11,12,16,20,24,25,27,30,31],
[0,1,3,11,12,16,22,24,25,27,28,29],
[0,1,3,11,13,14,16,18,25,27,28,30],
[0,1,3,11,13,14,16,18,26,28,30,31],
[0,1,3,11,13,14,17,18,24,27,28,30],
[0,1,3,11,13,14,17,18,24,27,29,31],
[0,1,3,11,13,14,17,19,24,25,28,31],
[0,1,3,11,13,14,17,19,24,26,28,30],
[0,1,3,11,13,14,18,24,25,27,28,30],
[0,1,3,11,13,15,17,18,23,27,28,30],
[0,1,3,11,13,15,17,18,23,27,29,31],
[0,1,3,11,13,15,17,19,23,25,28,31],
[0,1,3,11,13,15,17,19,23,26,29,31],
[0,1,3,11,13,15,17,19,23,27,30,31],
[0,1,3,11,13,15,18,23,25,27,28,30],
[0,1,3,11,13,15,18,24,26,27,29,30],
[0,1,3,11,13,15,18,24,27,28,30,31],
[0,1,3,11,13,16,17,21,24,27,28,31],
[0,1,3,11,13,16,18,21,24,27,30,31],
[0,1,3,11,13,16,18,23,24,27,28,30],
[0,1,3,11,13,16,18,23,25,27,29,30],
[0,1,3,11,13,16,21,24,25,27,28,29],
[0,1,3,11,14,15,16,20,26,27,29,30],
[0,1,3,11,14,15,16,20,26,28,30,31],
[0,1,3,11,14,15,17,18,20,28,29,30],
[0,1,3,11,14,15,17,18,21,27,29,30],
[0,1,3,11,14,15,17,18,21,28,30,31],
[0,1,3,11,14,15,17,18,22,27,28,30],
[0,1,3,11,14,15,17,18,22,27,29,31],
[0,1,3,11,14,15,17,18,23,27,28,29],
[0,1,3,11,14,15,17,19,21,25,29,31],
[0,1,3,11,14,15,17,19,21,26,29,30],
[0,1,3,11,14,15,17,19,22,25,28,31],
[0,1,3,11,14,15,17,19,22,26,28,30],
[0,1,3,11,14,15,17,19,22,26,29,31],
[0,1,3,11,14,15,17,19,22,27,30,31],
[0,1,3,11,14,15,17,19,23,26,28,29],
[0,1,3,11,14,15,17,19,23,27,29,31],
[0,1,3,11,14,15,17,20,22,25,27,31],
[0,1,3,11,14,15,17,20,22,26,27,30],
[0,1,3,11,14,15,17,20,23,26,27,29],
[0,1,3,11,14,15,17,20,23,26,28,31],
[0,1,3,11,14,15,18,19,22,26,30,31],
[0,1,3,11,14,15,18,19,23,26,29,31],
[0,1,3,11,14,15,18,20,22,25,26,31],
[0,1,3,11,14,15,18,21,25,27,29,31],
[0,1,3,11,14,15,18,22,25,27,28,30],
[0,1,3,11,14,15,19,22,25,27,28,31],
[0,1,3,11,14,16,17,19,21,23,30,31],
[0,1,3,11,14,16,17,19,21,24,29,31],
[0,1,3,11,14,16,17,19,22,24,28,31],
[0,1,3,11,14,16,17,20,22,24,27,31],
[0,1,3,11,14,16,17,20,23,26,28,30],
[0,1,3,11,14,16,18,20,22,24,26,31],
[0,1,3,11,14,16,18,20,22,25,30,31],
[0,1,3,11,14,16,18,20,23,25,29,31],
[0,1,3,11,14,16,18,21,23,25,28,31],
[0,1,3,11,14,16,18,21,24,27,29,31],
[0,1,3,11,14,16,18,22,24,27,28,30],
[0,1,3,11,14,16,19,21,23,25,27,31],
[0,1,3,11,14,16,19,22,25,26,28,30],
[0,1,3,11,14,18,19,22,23,25,26,27],
[0,1,3,11,15,16,17,20,23,28,30,31],
[0,1,3,11,15,16,17,21,22,27,30,31],
[0,1,3,11,15,16,17,21,23,27,28,30],
[0,1,3,11,15,16,17,22,23,27,28,29],
[0,1,3,11,15,16,18,20,22,24,30,31],
[0,1,3,11,15,16,18,20,23,24,29,31],
[0,1,3,11,15,16,18,20,23,26,29,30],
[0,1,3,11,15,16,18,21,23,24,28,31],
[0,1,3,11,15,16,18,21,23,26,28,30],
[0,1,3,11,15,16,18,21,23,27,30,31],
[0,1,3,11,15,16,18,22,23,26,28,29],
[0,1,3,11,15,16,18,22,23,27,29,31],
[0,1,3,11,15,16,19,21,23,24,27,31],
[0,1,3,11,15,16,19,21,23,26,27,30],
[0,1,3,11,15,16,19,22,23,26,27,29],
[0,1,3,11,15,16,19,22,23,26,28,31],
[0,1,3,11,15,16,19,22,24,26,28,30],
[0,1,3,11,15,16,20,22,23,26,27,28],
[0,1,3,11,15,17,18,21,23,25,28,30],
[0,1,3,11,15,17,18,22,23,25,28,29],
[0,1,3,11,15,17,19,21,23,25,27,30],
[0,1,3,11,15,17,19,22,23,24,26,28],
[0,1,3,11,15,17,19,22,23,25,27,29],
[0,1,3,11,15,17,19,22,23,25,28,31],
[0,1,3,11,15,17,19,22,24,25,28,30],
[0,1,3,11,15,17,20,22,23,25,27,28],
[0,1,3,11,15,18,19,22,23,24,26,27],
[0,1,3,11,15,18,19,22,23,25,27,31],
[0,1,3,11,15,18,19,22,24,25,27,30],
[0,1,3,11,15,18,21,22,24,25,27,28],
[0,1,3,11,15,19,22,23,25,26,28,29],
[0,1,3,12,14,15,17,18,20,28,29,31],
[0,1,3,12,14,15,17,18,21,27,28,30],
[0,1,3,12,14,15,17,18,21,27,29,31],
[0,1,3,12,14,15,17,19,20,28,30,31],
[0,1,3,12,14,15,17,19,21,25,28,31],
[0,1,3,12,14,15,17,19,21,26,28,30],
[0,1,3,12,14,15,17,19,21,27,30,31],
[0,1,3,12,14,15,17,19,27,28,30,31],
[0,1,3,12,14,15,17,20,21,25,27,31],
[0,1,3,12,14,15,17,20,21,26,27,30],
[0,1,3,12,14,15,17,20,25,27,28,30],
[0,1,3,12,14,15,18,19,21,26,30,31],
[0,1,3,12,14,15,18,19,25,28,29,30],
[0,1,3,12,14,15,18,19,26,28,30,31],
[0,1,3,12,14,15,18,20,25,26,28,30],
[0,1,3,12,14,15,18,20,25,27,29,30],
[0,1,3,12,14,15,18,20,25,28,29,31],
[0,1,3,12,14,15,18,21,25,27,28,30],
[0,1,3,12,14,15,19,20,25,26,27,30],
[0,1,3,12,14,15,19,20,25,27,29,31],
[0,1,3,12,14,15,19,21,25,27,28,31],
[0,1,3,12,14,16,17,19,21,24,28,31],
[0,1,3,12,14,16,17,20,21,24,27,31],
[0,1,3,12,14,16,17,20,24,27,28,30],
[0,1,3,12,14,16,18,20,21,24,26,31],
[0,1,3,12,14,16,18,20,21,25,30,31],
[0,1,3,12,14,16,18,20,24,26,28,30],
[0,1,3,12,14,16,18,20,24,27,29,30],
[0,1,3,12,14,16,18,20,24,28,29,31],
[0,1,3,12,15,16,19,21,23,26,27,29],
[0,1,3,12,15,16,19,21,23,26,28,31],
[0,1,3,12,15,16,19,21,24,26,28,30],
[0,1,3,12,15,16,19,22,26,27,29,30],
[0,1,3,12,15,16,19,22,26,28,30,31],
[0,1,3,12,15,17,19,21,23,24,26,28],
[0,1,3,12,15,17,19,21,23,25,28,31],
[0,1,3,12,15,17,19,21,24,25,28,30],
[0,1,3,12,15,17,19,22,24,26,28,30],
[0,1,3,12,15,17,19,22,25,28,30,31],
[0,1,3,12,15,19,21,23,25,26,28,29],
[0,1,3,14,15,17,18,20,22,23,28,31],
[0,1,3,14,15,17,19,20,22,23,26,29],
[0,1,3,14,15,17,19,21,22,23,26,28],
[0,1,3,14,15,18,19,21,22,23,26,27],
[0,1,3,14,16,18,19,21,22,23,25,27],
[0,1,3,14,16,18,19,22,23,26,29,31],
[0,1,3,14,16,18,20,22,23,26,28,31],
[0,1,3,15,16,18,19,22,23,25,29,31],
[0,1,5,6,16,17,19,21,22,26,29,31],
[0,1,8,9,11,12,15,23,25,28,29,31],
[0,1,8,9,11,14,15,21,26,28,30,31],
[0,1,8,9,11,14,21,22,24,25,27,28],
[0,1,8,9,11,14,21,23,25,27,28,30],
[0,1,8,9,12,13,15,19,25,27,29,30],
[0,1,8,9,12,13,15,19,25,28,29,31],
[0,1,8,9,12,14,19,22,24,25,27,29],
[0,1,8,9,12,14,19,23,25,27,29,30],
[0,1,8,9,12,14,20,22,24,25,27,28],
[0,1,8,9,12,14,20,23,25,27,28,30],
[0,1,8,9,12,14,21,23,26,27,29,31],
[0,1,8,9,12,15,19,21,23,25,27,30],
[0,1,8,9,13,14,19,20,24,25,29,31],
[0,1,8,9,13,14,19,22,24,27,29,31],
[0,1,8,10,11,13,15,23,25,26,28,29],
[0,1,8,10,11,13,16,23,24,26,28,29],
[0,1,8,10,11,14,15,22,25,26,28,29],
[0,1,8,10,11,14,15,22,25,27,28,31],
[0,1,8,10,11,14,16,22,24,26,28,29],
[0,1,8,10,11,14,16,22,24,27,28,31],
[0,1,8,10,11,14,17,22,24,25,28,29],
[0,1,8,10,11,15,16,22,23,27,28,31],
[0,1,8,10,11,15,17,22,23,25,28,29],
[0,1,8,10,12,13,15,17,26,28,29,31],
[0,1,8,10,12,13,15,19,24,26,29,31],
[0,1,8,10,12,14,16,20,25,27,29,30],
[0,1,8,10,13,15,18,21,22,24,26,28],
[0,1,8,10,13,15,19,21,22,24,26,27],
[0,1,8,10,13,16,18,21,22,24,25,28],
[0,1,8,10,13,16,19,21,22,24,25,27],
[0,1,8,11,12,15,16,19,20,23,30,31],
[0,1,8,11,12,16,19,20,23,24,26,31],
[0,1,8,11,13,16,19,20,22,24,25,27],
[0,1,8,11,13,16,19,20,22,24,26,31],
[0,1,8,11,13,16,19,21,24,26,29,31],
[0,1,8,11,14,15,19,21,23,26,27,29],
[0,1,8,11,14,16,18,21,23,25,28,29],
[0,1,8,11,14,16,19,20,22,23,26,31],
[0,1,8,11,14,16,19,21,23,25,27,29],
[0,1,8,11,14,16,19,21,23,26,29,31],
[0,1,8,11,15,16,18,21,23,24,28,29],
[0,6,8,10,12,14,17,19,21,23,25,31]])

In [ ]:
# Example: Construct the adjacency matrix of the dual fullerene C_60_1, i.e., dual fullerene (5,0)--nanotube
import networkx as nx
n = 60

for i in range(3):
    print(penta_indexes_matrix_C60[i])
    A = spiral_to_A(n,penta_indexes_matrix_C60[i],True)
    A_penta = A[penta_indexes_matrix_C60[i],:]
    A_penta = A_penta[:,penta_indexes_matrix_C60[i]]
    pentagon_patches = nx.Graph(A_penta)
    cc_sizes = [len(c) for c in sorted(nx.connected_components(pentagon_patches), key=len, reverse=True)]
    if len(cc_sizes)==2 and cc_sizes[0]==6 and cc_sizes[1]==6:
        print(i)
        print(cc_sizes)
        print('---------------------')